# Project: Comparative Analysis of the Data Scientist Role in the Top  Five Most Representative Countries within the Kaggle Community

## Table of Contents
<ul>
<li><a href="#intro">Introduction</a></li>
<li><a href="#wrangling">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
<li><a href="#conclusions">Conclusions</a></li>
</ul>

<a id='intro'></a>
## Introduction

In this research, the latest survey conducted up to the current date, known as the 2022 Kaggle DS & ML Survey [Kaggle](https://www.kaggle.com/datasets/kaggle/kaggle-survey-2017), will be analyzed. The aim is to utilize the data collected from the community and draw conclusions about demographics, industry involvement, and preferred technologies in the field of data science. Here are the question I plan on exploring:

> **1:** Carry out a data cleaning and preparation process for the 2022 Kaggle DS & ML Survey database.

> **2:** Identify the top five countries with the highest representation in the Kaggle community, focusing on the field of data science and considering levels of programming and machine learning expertise.

> **3:** Conduct a detailed analysis of the demographics and industries to which data science experts are affiliated, exploring factors such as gender distribution, age, education, geographic location, and areas of industrial specialization.

> **4:** Recognize the most used technologies and preferred techniques by data scientists.


In [ ]:
# exploration 
import pandas as pd
import numpy as np
# visualization
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go

In [ ]:
def group_question(min,max,v_v1):
    '''generates a list containing the names of individual or grouped questions depending on the numerical values of min and max.
    
    Args:
        min: minimum numerical value (x) representing the value of question Qx.
        max: minimum or maximum numerical value (x) representing the value of question Qx.
        v_v1: DataFrame summarizing descriptive information for each individual and grouped question.
        
    Returns:
        generates a list containing the names of individual or grouped questions
        group_question(9,9,v_v1)
        ['Q9'] 
        group_question(9,15,v_v1)
        ['Q6_1', 'Q6_2', 'Q6_3', 'Q6_4', 'Q6_5', 'Q6_6', 'Q6_7', 'Q6_8', 'Q6_9', 'Q6_10',
        'Q6_11', 'Q6_12', 'Q7_1', 'Q7_2', 'Q7_3', 'Q7_4', 'Q7_5', 'Q7_6', 'Q7_7', 'Q8', 'Q9'] 
    '''
    l=[]
    for i in range(min,max+1):
        con =f'Q{i}'
        val=v_v1.loc[con,('number_answers', 'count')]
        if val == 1:
            l+=[con]
            if min == max:
                break
            else:
                continue
        for j in range(1,val+1):
            l+=[f'{con}_{j}']
    return l

In [ ]:
def value_count_from_multiple_choice(df, v_v1,col):
    ''' dataframe for figures
    
    Works to obtain a DataFrame that indexes the unique value of all columns belonging to a 
    multiple-choice question 'col', containing columns for yes, no, and the percentage of users 
    who approve each unique value.
    
    Args:
        df: DataFrame containing data and columns.
        v_v1: DataFrame summarizing descriptive information for each individual and grouped question.
        col: numerical value (x) representing the value of question Qx.
        
    Returns:    
        a new DataFrame containing columns for yes, no, and the percentage of users 
        who approve each unique value.
    '''
    
    temp_df = df.copy()
    current_column_names=group_question(col,col,v_v1)
    # print(v_v1.T.loc[('question','unique'),f'Q{col}'])
    # print(current_column_names)
    renamed_column_names=v_v.T.loc['select',current_column_names].to_list()

    for c in current_column_names:
        temp_df[c] = temp_df[c].replace(np.nan, 0)
        temp_df[c] = np.where(temp_df[c] == 0, "No", "Yes")
        
    q_df = temp_df[current_column_names].copy()
    q_value_counts = q_df.apply(pd.Series.value_counts)
    q_value_counts.columns = renamed_column_names
    q_value_counts = q_value_counts.transpose()
    q_value_counts["% Users"] = round((q_value_counts["Yes"] / (q_value_counts["No"] + q_value_counts["Yes"])).round(4)*100,2)
    
    return q_value_counts.fillna(0).sort_values("% Users")

In [ ]:
def name_column_graph(tabla,symb):
    '''Creates a list with the indices of the 'tabla'
    
    Args:
        tabla: The result of the 'value_count_from_multiple_choice' function.
        symb: Some indices contain long names and can be separated by a symbol.
        
    Returns: 
        a list contains the indices of the 'tabla' for figures and less characters
    '''
    l=[]
    for i in tabla.index:
        a= i.split(symb)
        if len(a) > 1 :
            l+=[a[0]]
            
        else:
            l+=a
    return l
  

<a id='wrangling'></a>
## Data Wrangling

### Data exploration
The first step before doing anything with data is to locate the file (for this report a .csv file) in the directory I am working on and then read it through the built-in Pandas functions. In my case, I have `kaggle-survey-2022/kaggle_survey_2022_responses.csv` file stored in the same directory I’m working on.

In [ ]:
df = pd.read_csv('../input/kaggle-survey-2022/kaggle_survey_2022_responses.csv',low_memory=False)
print('Top 5 rows')
df.head(5)

In [ ]:
# Delete row 0, which contains the question name and corresponding answer for each column.
df_current = df.drop(df.index[0])
df_current.head(2)

In [ ]:
# return the rows and columns of the dataframe
print('Rows:',df_current.shape[0], 'Columns:',df_current.shape[1])


In [ ]:
# return the data types of each column
df_current.dtypes

From the above code excerpt, I have explored the data, shapes, and also it types. I have an overview of the kind of data I am dealing with. Now let’s describe the data through `describe()` function.

In [ ]:
# return a basic statiscal description
df_current.describe()

The `describe()` provides information about the data, thus it can be seen that all the columns are of categorical type and yield unique, top, and frequency results. It's useful information but I want to know quantity of responses corresponding to each question, empty values has this dataframe and procentage.

In [ ]:
aux=pd.isna(df_current).sum()
aux.name = 'nan count'

aux2= round(aux*100/df_current.shape[0],2)
aux2.name = 'nan %'

v_v = pd.concat([aux,aux2],axis=1)
df_quest = pd.DataFrame(columns=['question','select'], index=df.columns)

# columnas del Dataframe preguntas y  nombre seleccion
name= df.loc[:0,:].T
lm=name[0].str.split('?')
l_question=[]
l_select=[]

for i,v in lm.items():
    if ': (Select all that apply) - ' in v[0]:
        # len(pregunta) == 1 y contiene ': (Select all that apply) - '
        m = v[0].split(': (Select all that apply) - ')
        question=m[0].strip()
        select=m[-1].strip()
    else:
        # len(pregunta) == 1, solo pregunta 
        question=v[0].strip()   
        select='Selected Choice'
    
    if len(v) > 1:
        # len(pregunta) > 1 
        if ' - Selected Choice - ' in v[1]:
            #  contiene ' - Selected Choice - ' 
            select=v[1].split(' - Selected Choice - ')[-1].strip()
        elif len(v[1]) > 0:
            # string diferente de vacio 
            select=v[1].strip()
            if '- ' in select:
                select = select[2:]
            
    l_question+=[question]
    l_select+=[select]

In [ ]:

aux=pd.isna(df_current).sum()
aux.name = 'nan count'

aux2= round(aux*100/df_current.shape[0],2)
aux2.name = 'nan %'

v_v = pd.concat([aux,aux2],axis=1)
df_quest = pd.DataFrame(columns=['question','select'], index=df.columns)

# columnas del Dataframe preguntas y  nombre seleccion
name= df.loc[:0,:].T
lm=name[0].str.split('?')
l_question=[]
l_select=[]
for i,v in lm.items():
    if ': (Select all that apply) - ' in v[0]:
        # len(pregunta) == 1 y contiene ': (Select all that apply) - '
        m = v[0].split(': (Select all that apply) - ')
        question=m[0].strip()
        select=m[-1].strip()
    else:
        # len(pregunta) == 1, solo pregunta 
        question=v[0].strip()   
        select='Selected Choice'
    
    if len(v) > 1:
        # len(pregunta) > 1 
        if ' - Selected Choice - ' in v[1]:
            #  contiene ' - Selected Choice - ' 
            select=v[1].split(' - Selected Choice - ')[-1].strip()
        elif len(v[1]) > 0:
            # string diferente de vacio 
            select=v[1].strip()
            if '- ' in select:
                select = select[2:]
            
    l_question+=[question]
    l_select+=[select]

df_quest.loc[:,'question'] = l_question
df_quest.loc[:,'select'] = l_select

v_v=v_v.join(df_quest)

v_v.loc["Q37_12",'select'] = 'None'
v_v.loc["Q34_7",'select'] = 'None'
v_v.sort_index(ascending=True).T

# revision de valores mayores a 90% vacios
for i in v_v.index:
    count=i.split('_')
    v_v.loc[ i,'number_answers'] = count[0]

## split comlumna de dataframe str https://www.analyticslane.com/2020/10/19/separar-texto-en-columnas-con-pandas-en-python/#:~:text=Para%20separarlos%20podemos%20usar%20directamente,para%20obtener%20el%20siguiente%20resultado.&text=En%20el%20que%20se%20ha%20obtenido%20una%20serie%20de%20listas%20de%20valores.

v_v1=v_v.groupby(['number_answers']).agg(
    {'question':'unique','number_answers':'count','nan %':'mean', 
     'nan count':['min','max'],'select':'unique'})
v_v1.sort_values(('nan %', 'mean'),ascending=False).T

Summary table for 44 question:

1. question unique: Representation of questions in a single column. Note: there are single or multiple-choice answers.
2. number_answers count: shows the quantity of responses corresponding to each question.
3. nan % mean: indicates the average of blanks among the response columns (number_answers) corresponding to the question.
4. nan count:
min: indicates the minimum blank value in one of the response columns in the original dataframe.
max: indicates the maximum blank value in one of the response columns in the original dataframe.
5. select: unique: displays the response values corresponding to each answer in the original dataframe.

High missing values.

9 columns contain empty values ​​in a percentage less than 50%. I prefer to first select the data I am interested in.

###  Data cleaning

#### Selecting preferred data

Based on my research goals, I am not interested in all the columns in the dataset. In the below code except, I will only select the data I am interested in.

> Rename countries

> Remove Q4 'I do not wish to disclose my location'

> To do that, Dropping not desired columns.

> Removing unlikely or inconsistent responses

In [ ]:
# rename countries
countries ={'United Kingdom of Great Britain and Northern Ireland': 'UK','United States of America':'USA',
            'Iran, Islamic Republic of...':'Iran','Hong Kong (S.A.R.)':'Hong Kong'}
df_current.replace(countries, inplace=True)

## Remove Q4 'I do not wish to disclose my location'
i = df_current[df_current['Q4'].isin(['I do not wish to disclose my location'])].index
print(len(i))
df_current.drop(i, axis=0, inplace=True)

In [ ]:
print('Roles in question')
print(df_current.value_counts('Q23'))

In [ ]:
# select rows in Q23 equal to rol Data Scientist 

df_q = df_current[df_current['Q23'].isin(['Data Scientist']) ]
print( 'Number Data Scientist select:',df_q.shape[0])


In [ ]:
# Dropping not desired columns
'''
# Duration (in seconds)
# 5 Are you currently a student? Q9 academic research  Q10 use of machine learning for Q8 yes, Duration (in seconds)
# 9 published any academic research 
# 10 research make use of machine learning 
# 32 -1 cloud platforms that you are familiar 
# 33 - 5  cloud computing products 
# 38 -8 automated machine learning tools
# 40 - 15 monitor your machine learning models and/or experiments 
# 41 - 9 responsible or ethical AI products in your machine learning practices 
# 43 - 1 how many times have you used a TPU 
'''

l=["Duration (in seconds)","Q5","Q9"]
l +=group_question(10,10,v_v1)+group_question(32,32,v_v1)+group_question(33,33,v_v1)+group_question(38,38,v_v1) +group_question(40,41,v_v1)+group_question(43,43,v_v1)
print('Dropping not desired columns',l)
df_q.drop(l,axis=1,inplace=True)

# check the result
df_q.head(2)

In the process of data cleaning and quality assurance, only the role of data scientists was taken into account, leading to the exclusion of question **Q8 in case the respondent is a student**. Additionally, it's important to note that rows containing responses such as **'I do not wish to disclose my location' in question Q4**, which corresponds to country of residence, were removed. **Other row eliminations were performed according to:**
1. Responses corresponding to the age range (Q2) of 18-24 years and education options (Q8) 'Doctoral degree' or 'Professional doctorate' were discarded, as they indicated that respondents were currently pursuing or planning to enter this educational level within the next 2 years.
2. Rows were eliminated where age (Q2) fell within the 18-24 year range and programming experience in years (Q11) was over 20 years, as this scenario seemed unlikely and could have been an error in responses.
3. Rows were excluded where age (Q2) was in the 18-24 year range and experience in years using Machine Learning methods (Q16) fell within the 10-20 year range, as this appeared uncommon and required additional verification.
4. Empty values in the work industry (Q24) were discarded, as lack of information in this question would hinder proper analysis within the research context.

These cleaning criteria ensure data quality by removing unlikely or inconsistent responses, as well as empty values that could affect subsequent analysis.

##### 1. Responses corresponding to the age range (Q2) of 18-24 years and education options (Q8) 'Doctoral degree' or 'Professional doctorate'

In [ ]:
# remove rows in which the respondents declared their age to be 24 years or less 
# (categories 18-21 and 22-24 in question Q2) and at the same time declare that they 
# have a doctoral degree or a professional degree (in question Q8).

con1= df_current.loc[:,'Q2'] == '18-21' 
con2= df_current.loc[:,'Q2'] == '22-24'
con3=df_current.loc[:,'Q8'] == 'Doctoral degree' 
con4 =df_current.loc[:,'Q8'] == 'Professional doctorate'

df_c1=df_q[(con1 | con2) & (con3 | con4)]
df_c1.groupby(["Q2","Q8","Q23","Q4"]).count()

##### 2. Rows were eliminated where age (Q2) fell within the 18-24 year range and programming experience in years (Q11) was over 20 years, as this scenario seemed unlikely and could have been an error in responses.

In [ ]:
# We remove surveys in which the person filling out question Q2 indicates that has not 
# more than 29 years old (the three lowest categories) and at the same time declares 
# in question Q11 about the number of years of programming experience as over 20. 

con5=df_current.loc[:,'Q11'] == '20+ years' 
# NO existe
df_c2=df_current[(con1 | con2) & con5 ]
df_c2


##### 3. Rows were excluded where age (Q2) was in the 18-24 year range and experience in years using Machine Learning methods (Q16) fell within the 10-20 year range.

In [ ]:
# we exclude people who are also between 18 and 29 years old and declare that they 
# have been using machine learning methods for at least 20 years

con6= df_q.loc[:,'Q2'] == '25-29'
# Q16: Exp. ML methods
con7=df_q.loc[:,'Q16'] == '10-20 years' 

df_c3=df_q[ (con1 | con2 | con6) & con7 ]
df_c3.groupby(["Q2","Q8","Q23","Q4"]).count()

##### 4. Empty values in the work industry (Q24) were discarded.

In [ ]:
con8= df_q.loc[:,"Q24"].isna()
df_c4=df_q[  con8 ]
df_c4.groupby(["Q23","Q4"]).count()

##### Removing unlikely or inconsistent responses rows

In [ ]:
df_filtros = df_q[~(((con1 | con2) & (con3 | con4)) | ((con1 | con2 | con6) & con7) | con8 )].copy().reset_index(drop=True)
print(df_filtros.shape)

df_filtros.head(2)


#### Missing values 

In [ ]:

q2_i = ['18-21', '22-24', '25-29', '30-34', '35-39', '40-44', '45-49', '50-54', '55-59', '60-69','70+']

df_filtros.loc[:,"Q3"].replace({'Prefer not to say':'Others', 'Nonbinary':'Others','Prefer to self-describe':'Others'}, inplace=True)

q8_i = ['High School','No bachelor’s degree', 'Bachelor’s degree', 'Master’s degree', 
       'Doctoral degree', 'Professional doctorate','I prefer not to answer']
df_filtros.loc[:,"Q8"].replace({'No formal education past high school':'High School', 'Some college/university study without earning a bachelor’s degree': 'No bachelor’s degree'}, inplace=True)

q11_i=["0 years","Under 1 year",'1-3 years','3-5 years','5-10 years','10-20 years','20+ years']
df_filtros.loc[:,"Q11"].replace({"I have never written code":"0 years" , '< 1 years':"Under 1 year"}, inplace=True)

q16_i=["0 years", 'Under 1 year','1-2 years', '2-3 years', '3-4 years', '4-5 years','5-10 years',  '10-20 years']
df_filtros.loc[:,"Q16"].replace({"I do not use machine learning methods":"0 years", np.nan:"0 years"}, inplace=True)

q25_i=['0-49 employees','50-249 employees','250-999 employees', '1000-9,999 employees', '10,000 or more employees','No answer']
df_filtros['Q25'].fillna('No answer', inplace=True)

q26_i= ['0', '1-2', '3-4', '5-9', '10-14', '15-19','20+']
df_filtros['Q26'].fillna('No answer', inplace=True)

q27_i = ["1. Not Started" ,  "2. Exploration Stage","3. Generating Insights", "4. Started Models in Production",
        "5. Established Models in Production", "6. Not Known"]
Q27_replace = {
    "No (we do not use ML methods)": "1. Not Started" , 
    "We are exploring ML methods (and may one day put a model into production)": "2. Exploration Stage",
    "We use ML methods for generating insights (but do not put working models into production)": "3. Generating Insights", 
    "We recently started using ML methods (i.e., models in production for less than 2 years)": "4. Started Models in Production",
    "We have well established ML methods (i.e., models in production for more than 2 years)": "5. Established Models in Production",
    "I do not know": "1. Not Started",
    np.nan: "1. Not Started"
}
df_filtros['Q27'].replace(Q27_replace, inplace=True)

q29_i=['0-2,999', '3,000-9,999', '10,000-24,999', '25,000-49,999',         
        '50,000-79,999', '80,000-124,999', '125,000-249,999', '250,000-999,999','>$1,000,000']   
df_filtros.loc[:,"Q29"].replace({ '$0-999':'0-2,999',                    '1,000-1,999':'0-2,999',                 '2,000-2,999':'0-2,999',         
                                '3,000-3,999':'3,000-9,999',             '4,000-4,999':'3,000-9,999',             '5,000-7,499':'3,000-9,999',     '7,500-9,999': '3,000-9,999',            
                                '10,000-14,999':'10,000-24,999',         '15,000-19,999':'10,000-24,999',         '20,000-24,999':'10,000-24,999',          
                                '25,000-29,999': '25,000-49,999',        '30,000-39,999':'25,000-49,999',         '40,000-49,999': '25,000-49,999',        
                                '50,000-59,999': '50,000-79,999',        '60,000-69,999':'50,000-79,999',         '70,000-79,999': '50,000-79,999',       
                                '80,000-89,999': '80,000-124,999',       '90,000-99,999': '80,000-124,999',       '100,000-124,999': '80,000-124,999',     
                                '125,000-149,999': '125,000-249,999',    '150,000-199,999': '125,000-249,999',    '200,000-249,999': '125,000-249,999',
                                '250,000-299,999': '250,000-999,999',    '300,000-499,999': '250,000-999,999',    '$500,000-999,999': '250,000-999,999'}, inplace=True)

q30_i= ['0','$0', '$1-$99', '$100-$999','$1000-$9,999', '$10,000-$99,999', '>$100,000']
df_filtros.loc[:,"Q30"].replace({'$0 ($USD)':'$0','$100,000 or more ($USD)':'>$100,000'}, inplace=True)
df_filtros['Q30'].fillna('0', inplace=True)

df_filtros.loc[:,"Q37_12"].replace({df_filtros.loc[:,"Q37_12"].unique()[0]:'None'}, inplace=True)

df_filtros.loc[:,"Q34_7"].replace({df_filtros.loc[:,"Q34_7"].unique()[1]:'None'}, inplace=True)

#### Selecting demographics of data science experts, considering programming and machine learning experience. [4](https://www.kaggle.com/code/shivamb/data-science-in-2021-adaptation-or-adoption/notebook)

The classification of users was done based on the answers to questions Q16 and Q11, which are based on time of programming experience and machine learning experience, respectively. The categories into which users were classified according to their experience are presented below:

1. Newcomers: This group comprises those users who have ***no programming experience** or less than one year of experience in both programming and machine learning*.
2. The Programmers: In this category are users with ***programming experience** from 1 to more than 20 years*, but who have **no machine learning experience**.
3. The Apprentices: Users classified in this category have *at least one year and up to three years of **programming experience***, as well as *one year to three years of **machine learning experience***.
4. Transients: This group includes users who are somewhere in between. On the one hand, they have *3 to 5 years of **machine learning experience*** and *1 to 3 years of **programming experience***. On the other hand, there are users with *three or more years of **programming experience*** and *1 to 3 years of **machine learning experience***.
5. The Experts: In this category are users with extensive ***machine learning experience**, from 3 years to more than 20 years*, and ***extensive programming experience**, from 5 years to more than 20 years*.

These categories allow users to be identified and classified according to their level of programming and machine learning experience, which will facilitate a more detailed and segmented analysis of the data obtained in the survey.

In [ ]:
df_filtros1 = df_filtros.copy()

# Exp. ML 
Q16_di = {"0 years": "None", "Under 1 year": "Beginner", "1-2 years": "Learner", "2-3 years": "Learner", "3-4 years": "Experienced",
          "4-5 years": "Experienced", "5-10 years": "Veteran", "10-20 years": "Veteran", "20+": "Veteran"}
df_filtros1["Q16_new"] = df_filtros["Q16"].map(Q16_di).copy()

# Exp. progra
Q11_di = {"0 years": "None", "Under 1 year": "Beginner", "1-3 years": "Learner", "3-5 years": "Experienced",
          "5-10 years": "Experienced", "10-20 years": "Veteran", "20+ years": "Veteran"}
df_filtros1["Q11_new"] = df_filtros["Q11"].map(Q11_di)

# Cross table of those results
exp_ct = pd.crosstab(df_filtros1.Q11_new, df_filtros1.Q16_new, rownames=['Coding Experience'], colnames=['ML Experience'])
ct_order = ["None", "Beginner", "Learner", "Experienced", "Veteran"]
exp_ct = exp_ct.reindex(ct_order, axis="columns")
exp_ct = exp_ct.reindex(ct_order, axis="rows")

exp_ct

In [ ]:
conditions = [
    (df_filtros1['Q16_new'] == "None") & ((df_filtros1['Q11_new'] == "None") | (df_filtros1['Q11_new'] == "Beginner")),
    (df_filtros1['Q16_new'] == "None") & ((df_filtros1['Q11_new'] == "Learner") | (df_filtros1['Q11_new'] == "Experienced") | (df_filtros1['Q11_new'] == "Veteran")),
    ((df_filtros1['Q16_new'] == "Beginner") | (df_filtros1['Q16_new'] == "Learner")) & ((df_filtros1['Q11_new'] == "Beginner") | (df_filtros1['Q11_new'] == "Learner")),
    ((df_filtros1['Q16_new'] == "Beginner") | (df_filtros1['Q16_new'] == "Learner")) & ((df_filtros1['Q11_new'] == "Experienced") | (df_filtros1['Q11_new'] == "Veteran")),
    (df_filtros1['Q16_new'] == "Experienced") & (df_filtros1['Q11_new'] == "Learner"),
    ((df_filtros1['Q16_new'] == "Experienced") | (df_filtros1['Q16_new'] == "Veteran")) & ((df_filtros1['Q11_new'] == "Experienced") | (df_filtros1['Q11_new'] == "Veteran")),
    ]

classification_values = ['1.New Faces','2.Coders','3.Learners','4.Transitioners','4.Transitioners','5.Experts']
df_filtros1['classification'] = np.select(conditions, classification_values)
# 5 people are not classified as their experience with ML and coding does not match well
df_filtros1 = df_filtros1[df_filtros1.classification != "0"]
print(df_filtros1['classification'].value_counts().sort_index(ascending=True))

##### The top five countries with the highest representation in the Kaggle community, focusing on the data science role

In the first part of this report, the classification of experts, which includes those with extensive experience in programming and using machine learning, and who work in the data science industry, will be used. These data will be used to determine two groups, the first the five countries most represented in Kaggle.

According to their classification of expert experience, a descriptive analysis will be performed in three different categories, as follows:

* Demographic aspects: country, gender, age, area of study.
* Industry of participation: industry, company size, incorporation of machine learning methods, annual remuneration, on-the-job activities.
* Technologies used: programming languages, IDE integrated development environments, data visualization packages, cloud computing platforms, databases, business intelligence tools, machine learning frameworks, machine learning algorithms, favorite media sources on data science topics.

In [ ]:
# df_DS contiene  solo a los experotos en DS con todos los paises
conx1=df_filtros1.loc[:,"classification"] == '5.Experts'

df_DS=df_filtros1.loc[(conx1 ),: ].reset_index(drop=True)
print(df_DS.shape)
df_DS.head(2)
df_DS.groupby(['Q23','classification','Q4' ]).count().sort_values('Q2',ascending=False).head(10)

In [ ]:
q4_5top =  ['USA', 'India', 'Other', 'Brazil', 'Japan']
df_prue=df_DS[df_DS["Q4"].isin(q4_5top)]
print(df_prue["classification"].value_counts())
print(df_prue.value_counts("Q4"))


<a id='eda'></a>
## Exploratory Data Analysis

### > Identify the top five countries with the highest representation in the Kaggle community, focusing on the field of data science and considering levels of programming and machine learning expertise.

In [ ]:
class_all=df_filtros1.value_counts("classification").sort_index(ascending=False)
class_exp=df_prue.value_counts("classification")

fig = go.Figure()
fig.add_trace(go.Bar(x=class_all.index,   y=class_all.values,   marker_color='rgb(199,233,192)',name= 'All classification<br>and countries',text=class_all.values))
fig.add_trace(go.Bar(x=class_exp.index,   y=class_exp.values,   marker_color='rgb(0,68,27)',name= 'Expert classification<br>top 5 countries',text=class_exp.values))

fig.update_layout(barmode='overlay',title_text='All classification and countries.',yaxis_title="Number of responders")
fig.show()

In addition to categorizing experts based on their data science role, a geographical classification was also conducted focusing on the top 5 countries with the highest representation in the Kaggle community. These countries are **the United States, India, Others, Brazil, and Japan**, with a total of 548 surveyed users, **representing approximately 28.87% of the total users**, as shown in Figure.

In [ ]:
# df_DS
# df_prue

country_all=df_DS.groupby(["Q4",'classification'],as_index=False).size().sort_values('size',ascending=False,ignore_index=True)[:22]
country_study=df_prue.groupby(["Q4",'classification'],as_index=False).size().sort_values('size',ascending=False,ignore_index=True)[:22]
# country_comp=df_prue2.groupby(["Q4",'classification'],as_index=False).size().sort_values('classification',ascending=False,ignore_index=True)[:22]
fig = go.Figure()
fig.add_trace(go.Bar(x=country_all['Q4'],   y=country_all["size"],   marker_color='rgb(199,233,192)',name= 'All countries',
                    text =country_all['size'],textposition = "inside"))
fig.add_trace(go.Bar(x=country_study['Q4'],   y=country_study["size"],   marker_color='rgb(0,68,27)',name= 'Study countries',
                    text =country_study['size'],textposition = "inside"))

print(round((class_exp.sum()/class_all.sum())*100,2))
# anotations
fig.update_layout(barmode='overlay',yaxis_title="Number of responders",title_text='Expert classification, the darker green means the top 5 countries.')
fig.show()

This geographical classification provides an additional perspective on the distribution of expert data scientists in the Kaggle community, highlighting the relevance and participation of these countries on the platform. **Note**: "Other" is a special category that groups surveys from countries with fewer than 50 respondents, which may indicate that these are countries where the use of data science is in its early stages. This grouping helps identify those countries with lower representation in the Kaggle community, possibly due to lower development or adoption of data science compared to the more prominent countries on the platform.

### > Analysis of the demographics and industries to which data science experts are affiliated, exploring factors such as gender distribution, age, education, geographic location, and areas of industrial specialization.

#### **Gender**

The graph shows that men represent 86.32% of the total, while women and others make up 13.67%, the latter category being the minority. **Note:** The "Others" group includes non-binary people, those who described themselves uniquely and those who did not feel comfortable describing their gender.


In [ ]:

age_exp=df_prue.groupby(['Q4',"Q3"],as_index=False).size()

age_exp['avg_num'] = round((age_exp['size'] / age_exp['size'].sum())*100,2)
age_exp['avg']=age_exp['avg_num'].apply(lambda a: str(a)+ '%')

fig = px.bar(age_exp, x="size", y='Q3', color='Q3', facet_col="Q4",  text_auto=True,orientation='h', text='avg',#'texttemplate': '%{x}'
                   category_orders={"Q4": q4_5top,"Q3":['Man','Woman','Others']},
                   color_discrete_sequence=['rgb(0,68,27)','rgb(116,196,118)','rgb(35,139,69)'])
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Q4=", "")))
fig.update_layout(yaxis_title="Gender")
fig.update_xaxes(title_text="Number<br>of<br>responders")
fig.layout.legend.title.text = ''
fig.show()

age_exp.groupby(['Q3']).sum()

#### **Age** 

In the 30-34 age range, the highest participation is observed, comprising 23.9% of the respondents. **Approximately 75% of the participants are in the 25-44 age range, which shows a wide representation in this age group**. However, it is relevant to mention that the remaining 25% are distributed from 45 years of age to over 70 years of age. This diversity in age ranges highlights the presence of professionals from different generations in the data science community, enriching the variety of perspectives and experiences within the field.

In [ ]:
gen_exp = df_prue.groupby(["Q2"],as_index=False).size()
gen_exp['avg_num'] = round((gen_exp['size'] / gen_exp['size'].sum())*100,1)
gen_exp['avg']=gen_exp['avg_num'].apply(lambda a: str(a)+ '%')
color=['<75%']*1 +  ['>75%']*4  +['<75%']*5
fig = px.bar(gen_exp, x="size", y='Q2', color=color,  text_auto=True,orientation='h', text='avg',
                   category_orders={"Q2":q2_i},
                   color_discrete_sequence=['rgb(0,68,27)','rgb(116,196,118)','rgb(35,139,69)'])
fig.layout.legend.title.text = ''
fig.update_layout(yaxis_title="Age")
fig.update_xaxes(title_text="Number of responders")
fig.show()

In [ ]:
age_gen_exp=df_prue.groupby(["Q2",'Q4','Q3'],as_index=False).size()

age_gen_exp['avg_num'] = round((age_gen_exp['size'] / age_gen_exp['size'].sum())*100,2)
age_gen_exp['avg']=age_gen_exp['avg_num'].apply(lambda a: str(a)+ '%')

fig = px.bar(age_gen_exp, x="size", y='Q2', color='Q3', facet_col="Q4",  text_auto=True,orientation='h', text='avg',
                   category_orders={"Q4": q4_5top,"Q3":['Man','Woman','Others'],"Q2":q2_i},
                   color_discrete_sequence=['rgb(0,68,27)','rgb(116,196,118)','rgb(35,139,69)'])
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Q4=", "")))
fig.update_layout(yaxis_title="Age")
fig.update_xaxes(title_text="Number<br>of<br>responders")
fig.layout.legend.title.text = ''
fig.show()


In [ ]:
age_gen_exp[age_gen_exp['Q3'].isin(['Woman'])].groupby(['Q4','Q3']).sum().sort_values('avg_num',ascending=False)

In [ ]:
age_gen_exp[age_gen_exp['Q3'].isin(['Others'])].groupby(['Q4','Q3']).sum().sort_values('avg_num',ascending=False)

The following graph shows the age and gender distribution for each country. As expected, the participation of men is evident in all the countries that are part of the top 5. On the other hand, **the highest representation of women is found in the USA, with 6.93% of the total population of surveyed users, followed by India with 2.91%, other countries with 1.45%, Japan with 0.54% and Brazil with 0.36%**.
As for the inclusion of other genders, it is only observed in the USA and India, representing 1.09% and 0.36% respectively. These results reflect the differences in gender distribution in the data science community in each country and highlight the importance of promoting gender diversity in this professional area.

#### **Study**

The most representative level of study is the Master's degree, with 54.4% of respondents already holding or expecting to obtain this degree in the next 2 years. **Doctoral and professional degrees follow in order of frequency**. This indicates that the vast majority of respondents have at least one higher education study, reaching 92.3% of the total number of participants. These results highlight the high educational level of the data science community in Kaggle.

In [ ]:
study_=df_prue.groupby(["Q8"],as_index=False).size().sort_values('size',ascending=False)

d_ ={'Q8':'Other studies','size':study_[3:]['size'].sum()}
study_=pd.concat([study_[:3],pd.DataFrame(d_,index=[0])],axis=0)
fig = go.Figure(data=[go.Pie(labels=study_["Q8"], values=study_["size"],marker=dict(colors=['rgb(0,68,27)',  'rgb(0,109,44)','rgb(35,139,69)', 'rgb(65,171,93)']))])
fig.show()


In [ ]:

fig = ff.create_table(study_,colorscale=[[0,'rgb(0,68,27)'],[.5, 'rgb(247,252,245)'],[1, 'rgb(247,252,245)']])
## falta organizar colores y minimizar categorias para no tener tantas
fig.show()

#### **Industry**

The industry with the most data science representatives is Computers/Technology, with 176 respondents, corresponding to 32.12% of the population. It is followed by the Accounts/Finance industry with 55 representatives, equivalent to 10.04% of the total. The higher participation of data scientists in certain industries can be attributed to several factors:

##### Computers/Technology:
    
> **Abundant Data:** The technology industry generates a large amount of data, providing fertile ground for the application of data science techniques.

> **Constant innovation:** Technology companies are constantly innovating and looking for ways to improve products and services. Data science is essential to analyze and take advantage of these opportunities.
    
> **Intense competition:** Competition in the technology industry is high, which motivates companies to use data analytics to make strategic decisions.

##### Accounts/Finance:

> **Risk Management:** In the financial sector, risk management is crucial. Data science is used to assess and predict financial risks, which is critical to the stability of financial institutions.

> **Fraud Detection:** Fraud detection is essential in the financial industry, and data science techniques help identify suspicious patterns and anomalies.

> **Investment Optimization:** Financial firms use data science models to make more informed investment decisions.

In [ ]:

i='Q24'
title='Industry'
c1=df_prue.groupby([i],as_index=False).size().sort_values('size',ascending=False)
num=2
color=['']*num + [' ']*(c1.shape[0]-num)
c1['avg_num'] = round((c1['size'] / c1['size'].sum())*100,2)
c1['avg']=c1['avg_num'].apply(lambda a: str(a)+ '%')
fig = px.bar(c1, x=i,y='size' , text='avg', text_auto=True,color=color,
                color_discrete_sequence=['rgb(0,68,27)','rgb(199,233,192)'])
fig.update_traces(textposition='inside')
fig.update_layout(title=title,yaxis_title="Number of responders")
fig.layout.legend.title.text = ''
fig.layout.xaxis.title.text=''
fig.show()


In [ ]:
color=['rgb(0,109,44)','rgb(161,217,155)','rgb(35,139,6)', 'rgb(116,196,118)', 'rgb(65,171,93)' ]
i='Q24'
title='Industry split by top 5 countries.'
c1=df_prue.groupby(['Q4',i],as_index=False).size()
c1['avg_num'] = round((c1['size'] / c1['size'].sum())*100,2)
c1['avg']=c1['avg_num'].apply(lambda a: str(a)+ '%')
fig = px.bar(c1, x=i,y='size' , facet_col="Q4", color="Q4", text='avg',
                category_orders={"Q4":q4_5top},color_discrete_sequence=color,facet_col_wrap=1,
                height=800)
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Q4=", "")))
fig.update_traces(textangle=0)
fig.update_layout(title=title)

fig.layout.legend.title.text = ''
fig.layout.xaxis.title.text = 'Industries'
fig.layout.yaxis1.title.text = 'Number of<br>responders'
fig.layout.yaxis2.title.text = 'Number of<br>responders'
fig.layout.yaxis3.title.text = 'Number of<br>responders'
fig.layout.yaxis4.title.text = 'Number of<br>responders'
fig.layout.yaxis5.title.text = 'Number of<br>responders'
fig.show()

#### **size of the company **

The distribution of highly experienced data scientists in programming and machine learning working in companies with more than 10,000 employees compared to those working in smaller companies may be due to Financial resources and capacity, Abundant data, Technology infrastructure, Larger scope projects require the expertise of highly skilled data scientists, Diversity of Sectors.

On the other hand, smaller companies may offer a more agile environment and the opportunity to work on a variety of tasks due to smaller teams. Some data scientists prefer this more dynamic environment and the ability to have a direct impact on company strategy.

In [ ]:
color=[' ']+ ['']+[' ']*3
i='Q25'
title='Higher size of the company split by top 5 countries.'
i_i=q25_i
c1=df_prue.groupby([i],as_index=False).size()
c1['avg_num'] = round((c1['size'] / c1['size'].sum())*100,2)
c1['avg']=c1['avg_num'].apply(lambda a: str(a)+ '%')
fig = px.bar(c1, x=i,y='size' ,  text_auto=True,  color=color,text='avg',
                category_orders={i:i_i},color_discrete_sequence=['rgb(161,217,155)','rgb(35,139,69)' ])

fig.update_layout(title='Size of the company',yaxis_title='Number of responders')
fig.layout.legend.title.text = ''
fig.layout.xaxis.title.text=''
fig.show()


In [ ]:
color=[' ']*2 + ['']*1 + [' ']*3 +['']*1 + [' ']*4 +['']*1 + [' ']*3 +['']*1 + [' ']*5 +['']*1+ [' ']*3
i='Q25'
title='Higher size of the company split by top 5 countries.'
i_i=q25_i
c1=df_prue.groupby(['Q4',i],as_index=False).size()
fig = px.histogram(c1, x=i,y='size' ,  text_auto=True, facet_col="Q4", color=color,
                category_orders={"Q4":q4_5top,i:i_i},color_discrete_sequence=['rgb(161,217,155)','rgb(35,139,69)' ])
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Q4=", "")))
fig.update_layout(title=title,yaxis_title="Number of responders")

fig.layout.legend.title.text = ''
fig.layout.xaxis.title.text = ''
fig.layout.xaxis1.title.text = ''
fig.layout.xaxis2.title.text = ''
fig.layout.xaxis3.title.text = ''
fig.layout.xaxis4.title.text = ''
fig.layout.xaxis5.title.text = ''

fig.show()


#### **incorporate machine learning methods**

92.71% of Kaggle users who are data scientists classified as experts, indicated from the companies in which they work to have some kind of contact with machine learning methods. In addition, it was noted that 49.09% of these companies and industries already have an established model in production. These data reflect a high adoption and application of machine learning techniques in the business environment by highly experienced professionals in the field, which evidences the importance and relevance that the application of these methods is acquiring in various sectors.

In [ ]:
color=[' ']*4 + ['']*1 
i='Q27'
title='Higher status of machine learning methods into their business<br>by top 5 countries.'
c1=df_prue.groupby([i],as_index=False).size()
c1['avg_num'] = round((c1['size'] / c1['size'].sum())*100,2)
c1['avg']=c1['avg_num'].apply(lambda a: str(a)+ '%')

fig = px.bar(c1, x=i,y='size' ,  text_auto=True, color=color, text='avg',
                category_orders={i:q27_i},color_discrete_sequence=['rgb(161,217,155)','rgb(35,139,69)' ])

fig.update_layout(title=title,yaxis_title="Number of responders")
fig.layout.legend.title.text = ''
fig.layout.xaxis.title.text=''
fig.show()
c1

In [ ]:
# color=[' ']*2 + ['']*1 + [' ']*3 +['']*1 + [' ']*4 +['']*1 + [' ']*3 +['']*1 + [' ']*5 +['']*1+ [' ']*3
i='Q27'
title='Higher status of machine learning methods into their business split by countries'
i_i=q27_i
c1=df_prue.groupby(['Q4',i],as_index=False).size()
fig = px.histogram(c1, x=i,y='size' ,  text_auto=True, facet_col="Q4", 
                category_orders={"Q4":q4_5top,i:i_i},color_discrete_sequence=['rgb(161,217,155)','rgb(35,139,69)' ])
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Q4=", "")))
fig.update_layout(title=title,yaxis_title="Number of responders")

fig.layout.legend.title.text = ''
fig.layout.xaxis.title.text = ''
fig.layout.xaxis1.title.text = ''
fig.layout.xaxis2.title.text = ''
fig.layout.xaxis3.title.text = ''
fig.layout.xaxis4.title.text = ''
fig.layout.xaxis5.title.text = ''

fig.show()

#### **current yearly compensation (approximate $USD)**

Two significant peaks in the annual salaries of expert data scientists can be seen in the diagram. The first is in the $125,000 to $249,999 USD range, representing 30.11% of respondents, while the second peak is in the $25,000 to $49,999 USD range, comprising 25.55% of professionals surveyed. This data demonstrates that most data science experts receive competitive salaries, with a particular focus on the $125,000 to $249,999 USD or $25,000 to $49,999 USD ranges.

One issue that raises questions is the presence of salary ranges from $0 to $2,999 for experts in the role of data scientist. This may be discriminatory depending on the country. For example, in the United States, the salary range varies from $10,000 to $24,999 to an impressive range of $250,000 to $999,999. The highest peak is in the range of $125,000 to $249,999, with 151 users in the role of data scientist.

On the other hand, India and other countries that are part of Kaggle's top 5 have annual payout values in the $0 to $2,999 and $10,000 to $24,999 ranges, even though these users are classified as experts. In addition, it can be noted that, in some countries, even for experts with many years of experience the pay value stagnates in certain ranges, around $25,000 to $49,999.

**These variations in salaries may be related to the economic situation and hiring policies of each country, which could generate differences in the remuneration of expert data scientists. for examplo Average salary ranges USA: $ 59.860 , India: $ 7.000 , Brasil: $ 15.000 , Japón: $ 45.000 **


In [ ]:
color=[' ']*2 + ['']*2 + [' ']*5
i='Q29'
title="Yearly compensation (approximate $USD)"

c1=df_prue.groupby([i],as_index=False).size()
c1['avg_num'] = round((c1['size'] / c1['size'].sum())*100,2)
c1['avg']=c1['avg_num'].apply(lambda a: str(a)+ '%')

# brazil, india,japan, other usa

fig = px.bar(c1, x=i,y='size' ,  text_auto=True,  text='avg', color=color,
                category_orders={i:q29_i},color_discrete_sequence=['rgb(161,217,155)','rgb(35,139,69)'])
fig.update_xaxes(title_text=title)
fig.update_layout(yaxis_title="Number of responders")
fig.layout.legend.title.text = ''
fig.show()

In [ ]:
color=['rgb(0,109,44)','rgb(161,217,155)','rgb(35,139,6)', 'rgb(116,196,118)', 'rgb(65,171,93)' ]
i='Q29'
title="Yearly compensation (approximate $USD)"
c1=df_prue.groupby(['Q4',i],as_index=False).size()

fig=px.bar(c1,x=i,   y='size', color='Q4',height=800,
           category_orders={"Q4": q4_5top,"Q29":q29_i},
           color_discrete_sequence=color,facet_col_wrap=1, facet_col="Q4",
           text='size')
fig.update_traces(  textangle=0)
fig.for_each_annotation(lambda a: a.update(text=a.text.replace("Q4=", "")))

fig.layout.legend.title.text = ''
fig.layout.xaxis.title.text = title
fig.layout.yaxis1.title.text = 'Number of<br>responders'
fig.layout.yaxis2.title.text = 'Number of<br>responders'
fig.layout.yaxis3.title.text = 'Number of<br>responders'
fig.layout.yaxis4.title.text = 'Number of<br>responders'
fig.layout.yaxis5.title.text = 'Number of<br>responders'
fig.show()


#### **Activities at work**

On the most important activities for an expert data scientist, data analysis and understanding stands out as the main activity, according to 79.93% of the surveyed users, being the most frequent task. **The following activities are related to the use of machine learning:** prototyping for application in new areas, experimentation and iteration of ML models to improve existing models, and building or running a machine learning service to operationally improve products or workflows. These last activities are performed by more than 50% of the surveyed users.

In [ ]:
# graficas barras de preguntas con respuestas multiples mostrando porcentajes de solo expertos coun =  ['USA', 'India', 'Other', 'Brazil', 'Japan', 'Spain']m=['Brazil', 'Mexico', 'Colombia']
i=28

tabla = value_count_from_multiple_choice(df_prue, v_v1,i)
name ='Important activities of DS rol at work'
tabla.index =['None of these activities',
              'Other',
              'Research of machine learning', 
              'Build and/or run the data infrastructure',
              'Build and/or run a machine learning service',
              'Experimentation and iteration ML models',
              'Build prototypes applying machine learning',
              'Analyze and understand data']
fig = go.Figure()
color = ['rgb(199,233,192)']*4 + ['rgb(0,68,27)']*3 + ['rgb(199,233,192)']*1
fig.add_trace(go.Bar(x=tabla["% Users"],   y=tabla.index,   marker_color=color,name= '',
                    text =tabla["Yes"].values,texttemplate = "%{x}%<br>%{text}", orientation='h'))
fig.update_xaxes(title_text="% ")
fig.update_layout(barmode='overlay',title_text=name)
fig.show()

### > The most used technologies and preferred techniques by data scientists.

#### **Coding Language**

Compared to SQL and R, **Python** offers the following advantages for data scientists:

* Ease of use: Python has a simpler syntax than SQL and R, making it easier to learn and use.
* Flexibility: Python can be used for a wide range of tasks, including data cleansing, analysis and visualization.
* Scalability: Python can be used to train complex machine learning models on large data sets.

**SQL** is a very efficient language for accessing and manipulating data in relational databases. However, SQL is less flexible than Python and is not as suitable for data analysis and data science.

**R** is a very powerful language for statistics and data analysis. However, R can be more difficult to learn than Python and is not as suitable for general programming.

In [ ]:
i=12

tabla = value_count_from_multiple_choice(df_prue, v_v1,i)[-3:]
name='Coding Language'
color = ['rgb(199,233,192)']*(tabla.shape[0]-3) + ['rgb(0,68,27)']*3
fig = go.Figure()
fig.add_trace(go.Bar(x=tabla["% Users"],   y=tabla.index,   marker_color=color,name= '',
                    text =tabla["Yes"].values,texttemplate = "%{x}%<br>%{text}",textposition = "outside", orientation='h'))
fig.update_traces(textposition='inside', textfont_size=500)
fig.update_xaxes(title_text="% ")
fig.update_layout(barmode='overlay',title_text=name,    height=500, width=900,)
fig.add_layout_image(
    dict(
        source="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAOEAAADhCAMAAAAJbSJIAAAA81BMVEX///80erT/yh0nXIe/mBb/zB0jWoa9lhbDmxa7kQC+lg3KqkRQd5opYI0pYI7MohfarRk2ZY32+fv7+fLCnR707ts8bJLNsGK9lQC6jwBihKPIqEv/yAAATH0ZVYMxc6rY4ukubJ/uvRv2wxzs8PTktRrv8/Yvb6MndLEASnwLUYCXrMDhsxrU3OSvwM/H0t1xj6uKorh6lrCjtsfeypX59enp3La8ytfy69bYwojOsmbh0KHWvXrr4MH/5J3/6a10oMmdu9eUtdRhlMKvx95NiLyFq8/HpDPcyZHLq0f/0kj/7sH/4Iv/8c3/12P/9t7/34Wc3bx/AAANpElEQVR4nO2dd3faSBeHKbIoBlMNAoMMRsaJiAvYuOJNNpssjrNJ3u//aV4VpqnODMIz+PD7b8MeHT2+Zdqdq1Rqp5122mmnnSRRfWwYA0uGYYzrol8mKY2N87vr6axbatX0/sc+JjNbK3WvppOLy4GxnbjG+cW0W6qZlnRdtZQlpWatf9N13TT7eqt0Nbk7H4t+ZQYZF7NW1iLzYQXLQTXNbGlyLvrNaVS/u1L7FhwVm4dU7+tTySEtPN3UOeCg9H7pTjRFuMaT2np4riVNWRnrU53LNwMZDdE0AbrLrm8+jPFaNI9X9ZkZaj/nh3L5wNIQyP6PcjkKsSvX4FFvBRnQQitbUGfH6VAdH59ZuODPgEuvyeSpRs1vQIvtOALNR+pwEn8fVZ6Bw2fB8vCMmg3HJCFVXRrEK53EozedT0PSipI46kUfe6sDB++DrfUZ1ZoUc/I6FoNlly/95e9vX/9JczIeoAfqV6LpbE1M+EJD9xW/uX954+v6ZjQvBNNZqsM8qjqAH76j4LnkRDxDfpoVPyxemARgOo2HDi/iMfLTmTAyoC5IpGXn3T6QGZ7XUc9gLH4UnU9RnnHHiC+en/kArVhUZTHipUma0Lvw+cJpxDQa/QVH4jUgHLqEXp/6xksIQ1F0OoXzmdVExvsH5801yIhqVwgYVAvESzppQmBEtSbWTcGy0A3DBL00nQbLR1PsBBzMSQ9WhN88v3NnGjS10YWu9w0v4T/k72NuPuSm+lQMm6sBIByuXuvDJfH7v/wmTKelSDUD00NIRuIaUZiGgaiWhOGlggjT6QH8dT1ASNgShpcKJvzw1TXjYI0sIzehtcL//uXfL995V/lbQOhArkknPWEi2hG+LWG2TK0D2t1GyQhZRGnwLSbM0llxmwnpjLjNhOXtIax/5CI82B5CtJn4bgnHNVOl1zYSpurTUotaW0nIonFtR7gjlF3vn7AuO+H53WTW7ZbWkMS5tH4+qfVBLSy/pCUcTLP9hKrypCS87CZQUykx4aAUXpPHLYnWFvVZP3k+mWx4XkvaP11Js8a/3oCD2hpSAfoIb9tHfrUfX57veQFnfAv4OB3QFi56CNsFreiX5ujxmQfwigRU2SpGExBJeFNQwqUV8jeNtQDV7Do1lYkQNosRhDZk84kNcEpYcPjmeD7CTjSgohS1/TkD4MVH0XzMhDYjfTgOsGGw/Pb+yUmoFAvUnlqCgCrd6CwHoaLQIqKqWFBTKS1hzhKBeEsDaKCZjDgLhhDmKnu4MpnRaTWPMRY1mvEfFd+LBAwlzBCyMU8qiLG4jB8YB6j4XiQgJaFLeYoQtQW9CVVRWZSV0GIcIVctxPmpAefbQn2UjRBHLO7HEE51KXyUkdBGhH4anU/rsGR0uFWEmb0TaMRlJCGs3BZtQlbCzF4VIEYPitBJBUchO2EmA/w0OhKhk4pNpDyEaMzoRKRTWBMr3Ek91Zc0NsxUQK55DCe82GSVE5vAqNylJkRG1MInNrNVGKpc10CTFKyCntHbEEZixIBRkyYMwQ02fUJPCNOp1g4DHIO1Pd229CYFrpSYdwyEoxVhsRnmpnA0FD5WgDDMmgaDl+4BNy2EbWhMwGg4FA0InBRsl1ISQjd9CCHsShOGwElX1y0aq+3SXDWaELpp2KBf89xjEiZ4U3Z1KeiejhBl00JwIBoAUHiiKXuc9BYQfoomhG4aMjc9l2W8h/e5QZeTJ21FeBpDeBodiHBGI3i8R7e59dWbHa029XMnMYQwEIOXUFPPhUlBQne5YaOalQkVZRQThygQAwm7UiQa7D5+a3X5/RkQRg8WthHB7Dt4u6YkAyHeUwFchXsEYRiXSjN7n0AgBm1/w7Isgan0uIyOTExw9XAOfDQuDLHNjMAVlCF8zkb0qNHhvbwbGIYxfHaqUSJSje/C5BvjDYnWUeiOc6Op0DopNjVVAsb8y2SGw2O71ZXT+ope/r5YahZe1lxo1E5qpxrgpgGpBgyHayx/PXbgF9YK6xCeO8Vm0gyWaoKWF9drDoeJ4VmALXTddgmO8OMmNC4hTDUvfsLJWoTH3nZd/FL7WMMP6KNKPp4Pm9UEJVM4peEBTI7P8lCsgwgqM6GJQhsRzNuOEiUcRr4yk0x1irXxeUKANFGYQck06CQREKrMgOWE6sNU1eLDL7wjQCUXOyVdEYJ5W95ffcJrw2P/mzqtZRlld6Lt3hEdzBYYIE2acQjhEjGckNGGXg+13lTN1ljVKs2uPU0+5vsaAqQY7FeEcLjwD4h8NiQBdbN8dX05MMZ1NvmzgrXqRZVeuQolH7YIDhgQuQiJLnl6/+oyoUZ5t0tkQCumKIMwgw+I/tUFz3g4JOw3S6g11/ymqRGlevSAGOGN77kcc5ozDNDsDnyPbDBrfn/7sCwQfDkGC+JDvn9Swz4vxbKo6m2VM39+ePyxbDIqrxU00nyMgBihfzOKfW2B5qF6ljDg/GnfedWYqlAaWUmGBTCSkHl9iIJQb+EJ5r7d0RJgc/jidkj9iiA0wM0YyjU+1t+whAHOHwta9GszAFaYPDSOcFwjW3fFCU621RbWcuypmBxf/nSPGTCKkHGvDdvWRDHYOCok5J9KTvnEFoGuouKQcb8UmrCPFjuHWjIGzOWUCo/94giZ9ryhCbFmuM/rJxi7JjaXr5yO+PiiR3y2cwu05DVCAJ1XZVb108nIKRvlVNSsje3sCZpwgly0iOM5duATL51DCI+fAg7YWM4P4TE07Ek9z2NrAaU6WvNNuQkjVk9MZ8CeY2hLbWw1V+EOo/UJI1bATOf4vpHiiViPi+LDdjG0IMJr6lqMYxUM9sBHkQXZZsqJE67eIrimnb6eBjbfBHkG81GhgJkM2E0MrIuqU9dEAUJwxHdYkAQQDRbBRfvUdW0g0fTHHhNS74ltijDyiJShNhGsDMsrE0JA6j2xTRFGH3PT15d6uou+0JcSbFpR54cprEY4LhCzJOF+URYTnsTVfdHWeQPCLplncmL58DAMq92bUo6IJCEsWaI8PdkgIQzDsPpL2vsWJOGCsihr84CgUEHphACiT3LEZFOSEIah6DwDFxbFH2GEcLyIyTUkIcikVKe0GyWEk9Lw+7LITSNzDUHYALUEosMQ7mAELyxWqlHdziMI56BGWfhYcRpxxA0Fv1kRuZdBEB7SVvBunDAf76TEF5y2jRA5aeDaEGpCc9NZTsIqjZOmUmP4sb6I6/hSEmaonDSFTb8jZjYyElLd7HKFdY0IQ5SREJkw4naeK+xKfhiihIToJnDshXXie3EhzU1kJIS3ZKOvOrsiGgQHtXaSjxBFIVX7j3ELr+NSywdnniZR0hHuZRRoQqqeUYb3+67BlWsSEaLL6pRNagyVph2dNITIR8NvV/oQSxQN22QhRHmUsgmPo/qsvy2EGCBVIoW6i23sKQXhHtx+ohsLCTPGfTHdPVsTR+gcp55gfZSC6tejZUxrZgSke7bGTJjUH2I0Gp0QfaK06EVFiB0vZ7W+qQc1Edb77vcXGQn3MtV8QlLIXl/FgMpnSkveXU+vfH2gu9NBiocwg//Vk1RRO+QEjBMbIRqbkwbMc7cyTZgwH/2ivNKWm7Igs5duhK/YaTO3MN0mwqLWpJ/KbB9hUSssnzZowDUIAxoDM0rTtEKhuXjeLB83YfPH/po6ai+ebnmHwM0TxmxqSqUd4Y5Qfu0IgwmV5mObVouNzTg3Sqho9Cowr2ylIGRRh+vrANtEKDZqGVdPXIRKfosI+daH2hYR8q3xt4gwk6nwIG4Tob3XlqPWNhK6G7m0qmwlIYNQEdB7Jcy8J8KerXdM2Hv96+evn/+NPIySeOn6tYm911+rh332/AIIhc5pKPunRgH+RrtlfwIJi02BgKkUKNXnrqB9xZ82xxwVdZgLL2t+C8EvavGa8A/xuM8IMe4a01upTdleNAzwt+d56Cmo9zHjV8cS1gt1b8pgwp+e5/2HjBjbv/pt9AzvzHBdR+h5t61/AkJUZCF0sLCSKeiLx3dppud93i9ICEfD0D7ybySm5o30hIzVeJvU/Vp3ZD2pFCZTdMeHvpRrYzrCuhsyI/b+8jztd88FRN//EbuZaAu14MxxIO6RD/vjBSzmhZsQb1HJ2NcpwIiOCTFA6oLKjaqBfSmUveMAMSL+1fN+S03sjA0ISzYcXSN6n8FzGv/rWXwj/Ht40Tdg3k5Y2wi78wdjNPZ+/7Q5/nx+7Vl8VbyYS/B0BtMD/s1eh5EJspd5fX3NOHwKvt0oRRCu9EJ8ljiXq56wdnBxv9dIfnhTIkB04xky5qsjaks6paJVT1VfMeZLeG8uX6MoG/JkFNs3yP4fRqeVfM6zG65trhiPV/N93we07b5dK8oATvef7Q+mKl48pVg4kiSLEroJamiWczBPRyNyv9siHJ18qla8H71dGbAoVQgizdshTemcPmX5SqW6UqVSyfu+6Iv4tIWMBnT1fNSJ6tvmO3QJ4uscSReBhJ6PvF1IWVTUtPbmqmGT0uGiydcC066lvJHXP3E1bh81VkgLr7mQZpJGocbtQuvQ+qvlm53mg9zRF6j7p3ZTK0RTWnAFbfn4JqWim9H97eLIxvR1FnarfLVl++F5e+mAGvPnp4dHG7TQKVjqdApaftlevNzezyXYo0hUjfnh4f3h/N1x7bTTTjvttNX6P5DhGpVdY9bHAAAAAElFTkSuQmCC",
        xref="paper", yref="paper",
        x=1, y=0.73,
        sizex=0.1, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)
fig.add_layout_image(
    dict(
        source="data:image/png;base64,iVBORw0KGgoAAAANSUhEUgAAAOoAAAB7CAMAAABNcOyaAAAAwFBMVEX///8AcsYmd8e41DJ/ugAAX8Dx+f2Os94XdMcIbsQAbMRcjs6Ytd4AcMUAaMMAasPt8vlNiM0AZcKJvhB3tgCeu+Hl78j4+fx6odat0IXl7Pbm78K00hmeyCG9zui515jJ1uxnl9L7/PW+2FPO4YrF3H6rzird5fM3fcnF3qivx+aHqdrQ3vHy9+fs89zn8NDe6rbV5aLG3Gq51UDE23QAV77R45eUxFWCuzWu0XPT5KnZ6ceRw0ipz3zM4rSgymvTibGkAAAG90lEQVR4nO3ceUObSBgH4EAamRmOYVgRioqQWEEgR03Xutrq9/9WyzGcIYc1K0yW3x+Vgmnz5B2YYYCMRkOGDBlSzXQ6XywvLi4uLy9Xq/iPePFiOZ9Ou35fR810vvz++O1hNluvx1mu6M/1bDb7cfu4WrAPjpHPP2ZXScZbk25ePzwu2QVPL77/WO9ENsDr2e3lout3/QdZ3s7GhzJL73j2sGKrttPn9buZZXm/zbt+/4dnNftTJ8XeMtKOpw/jD0kT7HrVteKQLNYfhabY71079mf6cAxpbL3oWrI3q+NIx+NZ15K9+XYs6lXXkr15Pj7V6JCzGatYWhxLetvyj/chllcsbttZ7+6ur6//qiVecXe3RfpQDprsLkRbc2aWy6t6t5oQY9b5+fmXliSrE/Nd3VwdMalhB6DtOcNlWUfLor9JlV9ajc0kRc61V+PnykA4FD7fsyNnf7tq5a/Ldfx2764PITbA13fxx/RYHQTbuGfUiebWDpTLx59bmuzOnJ8//fNcO7WxAeobldPM+pFy/nL/613a+Jeffr82Bvo24GHvqBzyo8ba6fzla8rdIz5PmW/3G9MQagh6SeWgFnot2xavN7/ffv56SuvWSOx8+vn2dnP/0nJCbtg85PleUjkO+8KW/n6+WL68vN7f39zcfE0TL9zc37++vCzmW6YdbDMuaW+pHNyOfV8MW8ygvaVynKxoru19bNiqRhKPKLTH1KQZY98M2vbag6I6ksjDAtpvatKOISCC8/7aWoHp86Di7D01acgQTzQS2pGn7n95vG9anqO7vIY4vpneU2l1NeSLZqjbjqe219iwIicQQpfwGMlgw8kMNS0vwhjGCN/3ieiagiDpQRDogiSZLiEkXs0DWQatSsaoFJwGIqwkgUlkbjuPZWothxEHatcZqAN1oA7UgdpVBupAHagDdaB2lYE6UAfqQB2oXaWdmk6d/Q+oUFGSS4aypqE6V8YK4n2fR83p7XQeEW58Dn2noomo25HneZETmABWPgENu4ETJRt0F8EKyZfSiE1rv6kycmuXp8TcKkNfr8zyqyFfwICYrZMgS1QIg/pmF+UbhMYVHM+VWabKTWlO3fgI4hghYpiKNm4fy6gyaLuFzjBlZqmyT/dTI9IlPYjUnIrLmlpeeS1SdQGrVETfsxHKmqJhIOpGSsVi/tuOSXwiFrttxCxVoW/NxnJ2NIa+Lyc/82ILEAAeAI7kR+m0CTNJpe1U14ojcmLGJu1lQo12MCBv6R5klKpRqg2royQZ0mNSNCkujkN6a60hcmxSMT0AGyaujpIIvZ/JheUYGNPdVZfZpMokHw8FfomFbrYu4itXy7GerXQAo9Sy+7R0XqOtOH/DAayc2YDc77NJ5aBYjv5UiY71805VQNWTOMrzCKNUDldv+bbM9CQO0VKHuEqlbd2KT2fYpHKKW7mn0pCS8/Mt1OwzsQizVA761eFugPdUld0GnFgnxClbcagUVB3WqNlKj+GqJjsnEp18swplLGWLrUdgh2e4qkmng+Uw72Hj0QQdGXm1fpUelgNGhxCVaPloItCK0VLcmMuqZpsNl1VqZTw4cbLtQTz0p4uqVo6B6WBJ5RltwNgk+RCJ0ygmPsvJh8ajYEIxKD/X0Vk9s1H0kc4nN8bKMlZoqxVwvOvmp6cO4eNtwKcHqqyoJVWDefpPjQmebvoA8q5DtxNYnYVQbSkUirv7DaE2C+FIRQgL1PQOdc/Ke1YvPUtXKvOFlclgm69VtRKzz8/ZlNRqDAKz3kdvealNL12cBNUI6Yy3DKXmDfxGkB+PT4HqucXMS/MCx8gzi0tULFLjczgnf/zCUB1TqUy7cFBzo/LZDH1SXowDoqE24vaeymHNdwVJ1yXJ9DWFqwdpvinQa1TFVYw0pJnedzZp8eijF9WJtHIjhIiO82tW0AwT1N2JDcXYSW8OjjbCOpWX845Hb39m6oSoPMgHFMGeurJP5bFDX+zwO+t6AlRQPIZv+ydO5QExaciusp4CtexeTn1fPTQDtasM1IE6UAfqQO0qA/UUqaP/kto2pdphuLaZleNQFadrXD1Cc7bsiFXt2taI57+jrO+i9m1XHY0C1PaYycepsnjQlxh9anTlYOs7qJB4+//rT4/QOun7MSoi/frG3DwOjw8r7KFUAMP+td4sZy7Ax6MCQJyuRTviuNoBlT2EChDR+1rSLEaChXu0+6kc5PV+fYV3a9SQgN3aPVQZ+KbTteLAqI5AoLZdu4MKIPzI9z12ETXSCYqbMmx7gHULFcgY+aFjMdByNxIJLvEBwgjVK9ykgvQ2JuKGTr8PRLtz5tnpFzJymqYpeXB+DxZCGGPgu6YQ2BHLzCKGoVpebA7orVd6niAWepZ19ilN9l9HlsLW5BxsBwAAAABJRU5ErkJggg==",
        xref="paper", yref="paper",
        x=0.85, y=0.4,
        sizex=0.2, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)
fig.add_layout_image(
    dict(
        source="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBwgHBgkIBwgKCgkLDRYPDQwMDRsUFRAWIB0iIiAdHx8kKDQsJCYxJx8fLT0tMTU3Ojo6Iys/RD84QzQ5OjcBCgoKDQwNGg8PGjclHyU3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3Nzc3N//AABEIAJkAxQMBEQACEQEDEQH/xAAcAAACAgMBAQAAAAAAAAAAAAAAAQUGAgQHAwj/xABKEAABAwICBAcKDAQFBQAAAAABAAIDBBEFBiExQVESEyJhcYGxBzZCUlV0kbLB0RQWIzIzYnKSk6HS4SRDc4IVFzQ1oiZTVMLw/8QAGgEBAQADAQEAAAAAAAAAAAAAAAEDBAUCBv/EADERAQABAwEGBAUDBQEAAAAAAAABAgMRBBQxM1JxsRIhgdEFEzJRoSNBYRUikeHwQ//aAAwDAQACEQMRAD8A7igEAgEAgECJAFyQBzoNGfFaOK440PI2M0pgaMuP20QwX53uVwNSTG6117GNn2W+9MI134nWu11D+rQmB5Orao6fhM33ymBj8Mqf/Jm/EPvQNuIVjdVTL95MD1bjNez+ffpaCmBsRZjqmkcZFE8c12lMK3YMx07tE0T4zzcoJgSdPXUtT9BOxx3XsfQoNlAIBAIBAIBAIBAIBB5T1EVO3hTPDBzoIerxw3LaVlvrP9yuBEz1M1QbzSOd1q4R4lAigRQYlBjdAEoMSUCJQIlAiUGN92i3Og3qTGq2lsBLxjB4Emn89aYVYKDMFJUEMmPESHY75p61MCXDgdI0jeoGgEAgEAgECJsgisQxhkN46YB7/G8Ee9XAgZppJpDJK9znHaSrhHloQF0GN0CJQIlBjdAiUGJsgV0CKBFBigROhBiSgV0G/huM1WHkNa7jItsTjf0bkwq4YdiVPiEXDgdyh85h1tXkbYQNAIBAnEAXJsAgr2J4qZyYadxEW1w8L9lUROxUIlAiUCQK6DElAhdxs0EncEG5DhNfP82BzBvk5Kg3ostzH6WdjPstJTKthmWoPDqJj0ABMj1GXaK2l0x6XfsmQzl2g3S/fTI83ZaojqkqB/cPcmR4SZWj/lVbx9toPZZMjTmyxVj6KWJ/TdqZEbU4XXU1zLSycEeE0cIfkqjSugRKDOColppmzQPLJG6iEVdMDxmPEo+A+zKlou5m/nC8iWGpA0AUFfxvEC95poXclvzyNp3K4RDnmVCugRQYlAibIMoYZaiTi4WF7twCCbo8uggOrH3+ow+1TKpmnpIKYWgiYznA0+lQewcgLoC6AugLoC6AugLoC6DRrMKoq0fLQjheOzkkehMitYpluopgZaU8fENluWOrarkQWlVGUM8tPMyWF5bIw3aRsKKv+DYkzEqQSt0PbyZGbj7l5Egg0sVq/glI4tI4x3JagqZNzcnSvSESgV0GJKBE6kG9hmGSV7g8ngwA6X7+YKZVaaWkhpIxHAwNG3eelQe6AQcdxzNWP0uNYhBBikrIoqqVjGhjDwWhxAGlq71jSWKrdMzT54/n3ci7qLsVzEVfv/DR+OOY/K834cf6Vl2PT8nf3Y9pvc3b2HxxzJ5Xm/Dj/Spsen5e/ubTe5u3sXxyzJ5Xm/Dj/Smx6fl7+67Te5u3sPjlmTyvN+HH+lNj0/L3902m9zdvYvjnmTyvL+HH+lNj0/L39zab3N29h8c8yeV5fw4/0psen5e/ubTe5u3sXxzzJ5Xm/Dj/AEpsen5e/ubTe5u3sbc65ka4H/FpHW2GKOx/4pOi0/L3Xar3N2S1B3SsXpyPhdPTVTNtgY3ekX7Fr1/DbU/TOGWnW3I3xldcvZ1wrG3NhDjS1R0cTNYXP1TqPbzLn39Hctee+Pu3bWpoueW6Wxj2BMrWunpQ1lTrI1CTp5+dasS2FKeC1xa4FrgbEHWDuXob+A4icPxCN5d8k88GQc2/q96SOiAg6RpC8is5gqOMrRH4MQt1nWrCIwlUY3QK6DElBvYTQOr5yHXELLF5HYkqt7GNYwNY0BoGgBeRkgEAg4JmXvixXzyb1yvptPwaOjhXuJV1RqysZFAIEiEgEAgSBbVAtaDpXc8zhLPMzB8Wk4biLU07jpP1Hew9S5Ot0kRHzKPX/To6TUzP6dXomc4YaGObiELdDjwZbb9hXMh0VXuqOhZZqzV4PCXG74/k3dWr8rKSK/WScZVTP3vPaqjxQYkoFdBiSguuFUopKJkduWeU/pK8q3EAgEAg4JmXvjxTzyb1yvprHBo6OFe4lXVGFZWMIEiEUAgECRSKiEgEU2PfG9r4nFkjXBzXDW0jSCnlPlKecbnd8OnbmHKsMzgA6qp7uG5+3/kF81et/LuTT9netV+OiKlA07V4ZFryRUFsVXHsDmu9IPuSRpzXEsgOsPPamUeZKBFAkHtQMEldTtdqMg7UF6C8qaAQCAQcDzKf+o8V88m9cr6axwaOjhXuJV1RiysYRCQdI7neXsJxbAnVOIUMc83HvZw3E3sLWGtcnXai7bu+GmrEYdHSWbdVvNULT8S8ueSofS73rT2y/wAzb2e1yj4l5c8lQ+l3vTbL/MbPa5UBmfueUMlHLU4M0wVMbS4Q3uyTm06jzrZ0+vriqIuecdmve0dPhmaPKXKukEdK7DlhAkUXVR2LuVSmTKrWE34ud7fzv7VwfiEYvZ/h19FwldxBojr6lg1Nld2rUbaYygHufWcDdH/7JI9sVj4nEKhh8cuHXp9qiNQlUIoMboPajlEVXDIdTXgn0oL6F5U0AgEAg4FmbvjxXzyb1yvprHBo6OFe4lXVGrKxEgEHXO5P3sO86k7AuH8S43o62i4Xqui0G4EGMjmsjc55s1ouTzJ5/sPnOreySsqJI/mOlcW9BJIX1NMYpiHz0zEzMw8VUCKSDsvcshdDlNjnC3GTSOHOL29i4Wvqze9IdbRRMWvNWMQeJK+peDodK4j0rUhtrbkSm/gaid4+fIGjoA/dSR7Zppi2SKqaNBHAd07EgQJKqMSUCugxugu2B1orMPY5xBkZyX9I2ryqRQCAQCDgWZu+PFfPJvXK+mscGjo4V7iVdUasrESAQXPJ2daXL2Emino6iZxldJwoy0CxtvPMufqtHVeueKJbmn1VNqjwzCd/zSw/yZW/eZ71r/0yvmj8s+30cs/gf5p4f5MrPvM96f0yvmj8m308s/hXcy90CsxelfR0UHwOnkHBkPD4T3DdfYtixoKLc+Kqcz+GC9q6q48NMYj8qaNWq3Mug1MYCiEis4YpKiaOGBvDlkcGMbvJ1JMxEZkiMzh3aGFmXMqsgY4E0tOGA+M+2vrcvmbtyblyavu71qjwURS54A5xDWjhOJsBvK8vbqOD0QoMNgptrG8o73HSfzXmR71lMyqppIJByXi3RzoKLVQyU074ZRZ7TpXpHldAkGN0G3hWIPw6qEouY3aJG7x70VeaaeKohZLC4OY4XBC8j1QCAQcCzN3x4r55N65X01jhUdHCv8SrqjFlYggSBFQJAIEihAkBfTz6hzojp/c5yfJSPZjGKRFk5H8PA7XGD4TufcNnSuPrdXFUfLo3OnpNPNP99bbznizaqobQQPvFE75Uja/d1dq50N8sl4U6pq/h8zfkYTaO/hP/AGv6ehJkXxQBFwgicdwsVsQkhAFQwaPrDcrkU54cx7mPBa5psQdhVRjdBiSgRKDdwvFJ8OlvGeHETy4ydB9xTAuWHYnT4gzhQPHCA5UbtDm//b15VuA3QNBwHM3fJivnk3rlfS6fg0dHCv8AEq6o1ZmIkCUCQCBIBFZwQyTzMihZw5HmzWg2upVVFMZlYiZnELRhfc9x6tLXVMcNFEdPClkD3EcwaT+ZC07mvs07vNs0aO7Vv8l/y5kjCsDcKhzTVVY08dNbk/ZGodOtcy/rLl7y3Q3rWlotee+XnmTM7ImvpcMfwpTofMNTfs7ytaIbKu4HhE2L1fAbdsDdMsnijcOcqjpVLTxUtPHBAzgRsFmjcvI9kAgEERjWDMrwZIbMqANex3MferEin1EMtNK6KZjmPbrBVR4k3CDFAroHHI+J4kje5jxqc02IQTuH5pnhAZWR8c3x26He4qYVYaPGqCssI6hocfBfySoOJZn75MV88m9cr6XT8GnpDhX+JV1RizMRKBIqx4FkvFcdoG11FJRMic5zbTSPBuDbUGla17WWrNfgqyz2tNXdp8VLZre53mCkp3TAUlRwRcsglcXHoBaL+lY6fiFiqcecf91e6tHdiMqk4FpIcCCCQQdBBGsLdy1cEgTrEWOkKiVwjMeL4O8GgrpGsGuF54bHf2n2WKwXdPbux/dDJReuW/plaX50rMeiML3Cm4I5cUR+fz319S4+o0s2Zzvh1LGoi75bpbuA4BUYs8SHhRUo1y7+Zu9auWy6HQ0cFDTNp6aMMjbqG/nPOvI2EAgEAgEGpX0FPXxcCoYDbU4ax0FBU8TwCqpC58IdPFru0codIXrKIYlAroMboESisSUFMxA/x9Ve/wBM/tK+jscKnpDhXuLV1axWViJAKjpGQ8xw4Xl2Omkp5ZHCWQ3aRbSb7VwviEfrz6OxouDHr3T0mdY7ciheT9aQD2LSw2nPc6mOvrTikFK2nL/pmtde52O1DoXW0F//AMp9PZzdbZx+pT/38qySum0CQJEbFBVmirYKkRtk4p4cY3anjaD0hY7tuLlE0T+73RXNFUVQ+gsIrKbEMOp6uiINPLGHMtsG7q1dS+bromiqaat8O7RXFdMVR+7dXl6CAQCAQCAQKyDQr8Goa65miAkP8xmh37oK7W5UqY7uo5WTDxXclyuRCVVDV0pIqKeSO20t0enUqNa/OgV9xQUzEP8AX1P9Z/aV9HY4VPSHCvcWrq1ysrESBKiy4B/trftu7Vw9fxv8OvouF/nukD0rSbbB8Yna6ItLw8WLQL3XqmqaZiY3wlVMVRiVSr6Oagqn09QxzHt02cLEjYV9FauRcoiuP3cK5RNuqaZa6yMZIoQdA7lWYfgtY7Bap9oZ7vpyTqftb1jT0jnXN+IWfFT82P23t7R3cT4J9HVwbrjumaAQCAQCAQCAQCBEAixFxuQaNTg+HVBJlo4rnWWt4JPWEEdNlPDpL8B00Z5nX7UyONY7CKfG8RgYSWxVUjATrIDiF9LY4NPSHCvcWrq0SsrESA6VR0rIGXKfE8vx1U88oJleOCy1tDlwfiE/rz6Oxo+DHr3W6DKuExnlQOkt/wBx57FpZbSUpqOlpW2pqeKIfUYAgo3dWwMVVAzGKdvy1KODNbwo76+o9pXR+H3/AA1fLndPdo621mnx/Zykrsw5hIEoMo5HwyMlie6ORjg5rm6CCNRUmImMTuWJxud7yfjrMfwSGr0CcfJztHgyAafToPWvntTZmzcml2rF35lHiTiwMwQCAQCAQCAQCAQCAQfP2Z++XFvPJvXK+l0/Bp6Q4V7i1dUYsrEECRXZe5V3oRf15fWK4XxDjz6dnX0fBj17rgtJtBB5zxMngkilaHRyNLXNOog6CFYmYnMJMRMYl8/5mwh+BY3U0L7ljDwonHwozqPs6Qvo7F2LtuK3DvW5t1zSiisrGEAgsmQcwf4DjTeOdajqrRTk6m+K7queolamssfNt+W+Gxprvy6/PdLugIsLFcF2TQCAQCAQCAQCAQCAQfPmZ5GDMuLXc0fxs2s/XK+m08TNmjo4V6Y+bUizIzx2/eWXEsWRxjPHb6UxP2BxrPHb95MT9jLs3cpc12UIS0gjj5dI+0VwfiHHn0djR8GPXuuK0m0EAdIQUfuo4CcQwYYhTsDqmiu5w2uj8IdWvqW9oL3gr8M7paesteOjxRvhxvjY9j226V3MS5Q42Px2+lMT9gcYzx2+kJiQjJHtcLdIUxI7J3MMyDFsK+AVEofV0QDb30vj2HpGo9S4musfLr8UR5T3dXSXfHTid8LytFuBAIBAIBAIBAIBAIKpUf6qo/qu7Stqn6PSGtP1PNy8KSgbdY6VY3pVulP4J/o/73dq8XvrZLX0pBY2UIBB41X0En2D2JH1QlX0yqq2rn1NWjcy2Lw9k3WpIZXkbuC/7gf6ftCyXeFBa4krCtdsP//Z",

        xref="paper", yref="paper",
        x=0.45, y=0.09,
        sizex=0.1, sizey=1,

        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)
fig.update_xaxes(range=[0, 110])
fig.show()

#### **Integrated Development Environments IDE’s**

There are several reasons why a data scientist might use **Jupyter Notebook** more than Visual Studio Code and JupyterLab.

* Interactive user interface: Jupyter Notebook has an interactive user interface that allows data scientists to run code, visualize data, and explore models interactively.
* Integrated documentation: Jupyter Notebook allows data scientists to easily document their work, which is important for reproducibility.
* Easy collaboration: Jupyter Notebook allows data scientists to easily share their work with others, which is important for teamwork and collaboration.

**Visual Studio Code** is a general-purpose code editor that is suitable for a wide range of tasks, including data science. However, Visual Studio Code is not as well suited for data science as Jupyter Notebook because it does not have an interactive user interface or integrated documentation.

**JupyterLab** is an extension to Jupyter Notebook that offers a more advanced user interface. However, JupyterLab is not as suitable for data science as Jupyter Notebook because it is not as easy to use and does not have as many collaboration features.

Overall, Jupyter Notebook is a good choice for data scientists because it is a natural collaboration, exploration, and documentation tool.

In [ ]:
i=13

tabla = value_count_from_multiple_choice(df_prue, v_v1,i)[-3:]
name='Integrated Development Environments IDE’s'
color = ['rgb(199,233,192)']*(tabla.shape[0]-3) + ['rgb(0,68,27)']*3
fig = go.Figure()
fig.add_trace(go.Bar(x=tabla["% Users"],   y=tabla.index,   marker_color=color,name= '',
                    text =tabla["Yes"].values,texttemplate = "%{x}%<br>%{text}",textposition = "outside", orientation='h'))
                    
fig.update_traces(textposition='inside', textfont_size=500)
fig.update_xaxes(title_text="% ")
fig.update_layout(barmode='overlay',title_text=name,xaxis_range=[0, 88],    height=500, width=900,)
fig.add_layout_image(
    dict(
        source="https://avatars.githubusercontent.com/u/22800682?s=280&v=4",
        xref="paper", yref="paper",
        x=1, y=0.73,
        sizex=0.1, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)
fig.add_layout_image(
    dict(
        source="data:image/jpeg;base64,/9j/4AAQSkZJRgABAQAAAQABAAD/2wCEAAkGBxIPEBIPDxASFRUTExYVFRYXDQ8XFRcSFRgYFhUYFRUYHSggJCYxHRUYITEjJikrMi86Fx83ODMuOCgvLisBCgoKDg0OGxAQGi0mICYtNTU4Ny0tLS0yNS8tLS0tLS8wLy0tLS01Li0tLS0tLTUvLS01NS8tLS0wLS8yLy0vMP/AABEIAOEA4QMBIgACEQEDEQH/xAAcAAEAAgMBAQEAAAAAAAAAAAAAAQcCBQYEAwj/xAA9EAACAQIDBQMKBAUEAwAAAAAAAQIDEQQGMQUhQVFhEiIyE1JxcoGRobHB8DNzstEjNEJiohWSwuEkQ4L/xAAaAQEAAgMBAAAAAAAAAAAAAAAAAgUBAwQG/8QAMBEAAQMCAwUHBQEBAQAAAAAAAAECAwQREiExBSJRYfBBcYGRsdHhEzKhwfFSIxT/2gAMAwEAAhEDEQA/ALxAAAAAAAAAAAAAAAAAAAAAAAAAPnUqKMXKTSSTbbdkktW2AZN23s4LMmdX2nSwTVludWyd3/Ynut148Ob8Gbc1PEXoUG40dG96dT9o9OPHkcoyxgpbbz08CtqKu+7Gvj7e5uKOZK/a/iVpv+7tyTXxOn2fmatT3VP4kers/ZL97leyOjoPuR9VfI7kjY9MLkRTgWV7FxNcpY2ztsUcRZRlaXmvc/ZwfsNmVSbjZ2Y61Gyk/KR5Nu/sevvuccuz11jXwX3O2HaKaSJ4p7e3kd8DU7O27Rr2Sl2ZP+mW536PRm2K5zHMWzksWTHtel2rdAACJIAAAAAAAAAAAAAAAAAAAAAAAAAHj2jj6eHpyq1pKMY+9vgkuL6BM8kMKts1PtisRClCVSpJRjFXbb3JFXZpzPPGS8nC8aKe6PGbWkp/RfXTzZkzFUx09940ovuU7/5T5v5cOLemLSnpsG87X0Kmpq/qbrdPX4FwwQ2dhyGEmdFh33I+qvkc3JnR4fwR9VfIzHqpCXRD7XFyAbjSDa7O2/WoWXa7cfNk293SWq+XQ1QIvY16Wcl0JMe5i3atlO92dmGjWsm/Jy82TVm+ktH8H0N0VQbvL23JUZxp1JN0m7b34OTT5c19utnoLJePy9i0p9o3VGyJ4+/ud4ACsLaygAAwAAAAAAAAAAAAAAAADUbf25SwdPt1HeTv2IJ96T+i5vh6bJ5RFctkMOcjUuuh9tr7VpYSk6taVlokvFKXCMVz+2VRt7blXG1O3U3RXggn3Yr6vm/ktx59r7Uq4uq6taV3pGK8MY8or7ueMtoKZI81zUpqmpWVbJoTcXIB0nKTchgxYBEmdHh/BH1V8jm5HRYfwR9VfIlHqpGXRD7AgG40kggAEggAGx/1yt5z94NUCP0I/wDKeRs/9Ev+lLdAODzXm7xYfCy6Sqp++MH9fdzPPQwuldhaejmmZC3E46rE7bw1KXYqYimpLc12ldPrbT2nsw9eFSKnTlGUXo4yTT9DRSRsNjbYq4OfbpS3PxQfhl6Vz6r/AKO9+zUw7rs+ZWs2muLebly6zLjBp9hbepYyN6btJLvQfij1XNdV8NDcFa5qtXC5LKWrHtemJq3QAAiSAAAABzGbM0wwUfJwtOu1ujfdBPSU/otX01JMY564WkHvaxuJ2h6cy5kp4KG+0qsl3Kd/8pcl8+BU+0MdUxNSVatLtSl7kuEYrguh8sTiJ1pyqVZOU5O8pNb2/vhwPmXEECRJzKWoqHSry6/JIIBvNBIIABJDBDAMZHR4fwR9VfI5+hQnVnGnShKc5O0YpXb++fA6PyLp/wAOXiguzLffvR3Pf6USi1VCEqZISCAbjSSCAASCAAYgAmYN1nLM8pSnhqDtGLcakk98mt0orkuD5+jXjj6YunKFScJ+KMpKXrJtP4nxuc8MTY2Ijf6b55XSPVzv5yMgY3FzaaT64evKnJTpycZRd1JOzTLDy1m+Ne1LEWhU0UtIzf0fTR8ORW9wzRPTsmSzvM3wVD4Vu3Th2F7grPLWcZ0LUsS3Olopaygv+S6arhfcixcNXhVgqlOSlGSumt6ZST074Vs7zL+CpZMl268O3rrU+4Bwecc5Kl2sNhZXnvU6i0hzjDnLm+Hp0hHG6R2FpOWVsbcTj2ZuzdHCp0KDUq3F7nGn6ecuS9r5OsKtWU5Oc5OUpO7bbbberbPm3fe9XvbvvberZJcQwNiSyFJNO6VbqZXFzEG40mVxcxABlcXMRcAyubLYWwq2On2aKtFPv1HfsR/d9F8FvNxlbJk8TatiO1To6paTqLpyXXV8Nblm4TCwowjTpQUIRVlFKyOKerRm6zNfwdtPSK/efkn5X2NfsDL1HBQtSV5td+o/FL9l0Xz3nDbW/HrfmT+bLSKs2t+PW/Mn+pjZqqrnKvL9k9pNRsbURMrnlBALYpiQQACSTEAyAYgEDs82ZWWKTr0bKsl6FUS4Pk+T9j5qtqtNwk4yTjKLs00001qmi9jms05ZhjI9uFoVkt0t1pJaRn9Hw+BS0lbg3JNPT4L6sovqb8evDj8+pVgM8TQnSnKnUi4zi7Si9U/vjxPmXJRkggGQSbTYe3q2Cn2qbvFvvU232Zdej6r46GqBFzUcmFyXQk1zmrdq2U6zNWeZVKao4VTpqcf4k3ZS3/0Qa09b3czhEz3SSe5nkrUXHet6+RzNgSJLM0Oh1Qsq3fqSmLnyUjNMkYMwYXFzJgzBhc2WxNi1sbU7FCO5eOb3Qgv7nz6Le/iYcqNS6mURXLZEzPFRoyqSjTpxcpSdoxSbbfRFk5WyRGj2auMUZ1NY09YQfC/nP4LruZu8u5bo4GPcXaqNd+o0u0+i5LovbfU3pVVFYr91mSflS2p6JGbz81/Ce6gAHEd4Kr2s/wCPW/Nn+plqFVbW/HrfmT/Uy02Z9zvAqtq/Y3vPLcXALcpRcXAAFxcAAxBAMmC4gAeTPYGhzHl6njYb+7Uiu5O3wlzXy4FV7QwNTD1JUq0ezKPua4OL4rqXkajb2w6WMp9ipukr9iaXei/qua4+mzXdSVixbrvt9OuBwVlEk2837vXrj5lOA9m19l1cJVdKtGz1jJeGUecX92PEXjVRyXTQoXIrVsqWUkEAyYuSLkAC58K2Hvvj7jz3tuZ7z51aSl6eZrdHfNCbZLZKeZMXIjQm5qnGLlKTtFRTbk+SSLKylkNU+zXxyUp6xpbnCPr8JPpouvDllmbEl3HXFC6VbNNDlXJtTF2q1r06Gqdu/UX9iei/ufsvqrSwGBp4emqVGChCOiXzb1b6s9YKiad0q56cC5gp2QplrxAANJvAAABVO1fx635s/wBTLWKo2r+PW/Nn+plpsv7neBVbV+xveeUAFuUgABkAAGAQDG4MmC5AAeTPYg1+19qUsJSdavLsxW5L+qUuEYri/vQ+O39uUsDS8rWe97oQTXanLkl83wKb29tqrjqvlaz0uoQXhhHlH6vj7kuqnpllzXTrQ5KmqSJLJm7rU+2ZMy1sdV8pLuwjfydNPdFPi+b6+48dKqpK6PCxGTTui4j/AOaWRMikkvIuJy5myB8aNZS9PI+h0pmcyoqamQMQZMGQMQAffBYypQqRq0puM4vc19Vo10LNyxm6ni7U6tqdbld9iXqt8ej9lyqwc9RTMmTe149dnI6aeqfAuWacOtF7i/gVxlfOsoWo4xuS0jV3uS9fmuuvO+qsKjUjOKlCSlGSummmmno00UM8D4Vs74PQQVDJm3b8ofUAGk3gAAAqfa/49b86f6mWwVNtf+YrfnT/AFMtdl6u8P2VW1vsb3nlBALgoyQQACQQAADEEjBcxoczZkpYCn2p96cl3Kae+T5vkuvzPPm3NdPAQ7MbTryXchfcv7qltF01fvap7HY2piKkq1abnOTu2/gkuC5JHnaakWTedp6/B6Sqq0j3W/d6fJ9trbUq4uq61efak9OEYx4RguC+3d7zyEAt0RESyFKqqq3UkwZkBYHzvZ3R7KFftbuP3oeRns2RsmtjKqpYeDlLVvSMF505cF8XwuMWDPsMqzHkmp9QdJmHJ1XCU41Yy8rFRXlGoWcZcXbzevDiczc2xyNkbiat0NUsT4nYXpZTIGNxcmazIGNxcAyN3lzM1bBSt46Tfeg3u6uL4P4PjzWiuLkXxte3C5LoSZI5jsTVspeGyNr0sXT8pQldf1Jq0ovlJcPrwNiUPs7aFTDVFVozcZLlo1ykuK6Fo5YzZSxlqc7U63m37s+bpt/LVddSkqqF0W8zNvp8cy+pK9s267J34Xu58jpwAcBYAqXa/wDM1vzZ/qZbRUu1/wCYr/nT/Uy12Xq7w/ZU7W+xveeUggF0UZIIABIIAABAMGLmg2njJV69WtNvtVJyk78N+5exWXsPNc63PuVp4WrPE005UKk3Jtf+uUndxl0u9z625X5G5xxPa9iK3Q7pWOY9Udr1n4k3FyLi5sNZNxciKbaSTbbskk223okixso5A8NfHrrGjf3Oo/8Aj7+KNUszYm3cbYYXyus057KuUK2PanK9OhffUa3ytqoJ68r6LraxbmytlUcJTVKhBRit75yfnSerZ7IQUUkkkkrJJWSS0SRmU09Q+Zc9OHWpeQUzIUy149aGLV9zK3zlk10+1icJG8dZ0kt8ecodOnDhu3KygRgnfC7E3+mainZO3C7+H58uCxM5ZN7faxOEj3tZ0kvFzlBc+cePDfuddnoYJ2TNxN/h5uenfC7C7+9dvAAA32NAAAsASm07p2a3p8U1pYgCwO+yrnhrs0ca78I1ePRVP39/Flg05qSUotNNXTTumno0ygDf5YzXVwTUHepRb3wb3xvq4Ph6NH01Kuq2ejt6LJeHZ4cPTuLak2krbMlzTj2+PHv17y5Co9sP/wAmv+bP9TLN2VtSli6aqUJqS4r+qL5SXBlZbY/ma/5s/wBTNey0VHvRdTbtVUWNipx/R5QQC4KMkEAAkEAAgAEgXJVpqcXGSTi0000mmnuaaZU+d8lPC9rE4ZOVHWUd7lT69V14ceZbgPKwzuiddD1s8DZm2cfmy57Nl7NrYuqqNCDnJ+6K86T0S+9S3sZkPAVZufkXG7u1CpOMf9q3L2WN1svZVDCQ8nh6cYR1drtt85Se9+1lg/aDcO6i35lazZr8W8qW5Gkynk+lgUqkrVK9t82t0b6qmnpyvq+idjqgCre9z1xOW6lqxjWNwtTIAAiTAAABxeccnrEKVfDpKrrKO5Kp+0uvHjzO0BsilfE7ExTXLEyVuF6ZH58nFxbjJNNNppppprc00zEtnN2VIYxOrStGulra0ZpaKXXlL6aVTiaE6U5U6kXGUXaUWrNM9FTVLZ23TXtTrsPNVNK+ndZc07F67TAEA6DluSCABckhgiQMnp2XtSthKiq0JuMlqtYyXmzjxX2rG4rYl1ZOq0k599paJyfaaXvOZmdBQ8EfVXyINamJV7bEnPdhRt8r/k+gIBsNZIIABIIAABiDILrAB489oAAAAAAAAAAAAAAADnM15Yp46F1aNWK7k7f4zXFfFcOKfRgkx7mORzVspCSNsjVa5LoUBtDBVMPUlRrQcZx1T+DT4rqee5deY9gUsdT7M+7OPgmlvi+vNc18nvKg2tsyrharpV42kt6a3xlHhKL4r7Z6Kkq2zpZcncPY83V0boFvq3j79ZnjuLkA6ziJuYtkmMmDJhM6Ch4I+qvkc9M39DwR9VfIi3Uy7Q+gIBMgSCAASCBcAA9n+l1fMZJH6jOJs+k//KlvgA8iexAAAAAAAAAAAAAAAAAABq9vbGpY2k6VZdYyXihLnF/TibQGWuVqoqLZTDmo5LKmRRW39iVcDV8lUW53cZpPsyjzXXmuHub1Vy/Nq7MpYqlKjWj2ov3p8HF8GU9mfLdXA1LS71OT7k0tz6S5Pp7i/o61Jkwuyd69cDzlbQrDvtzb6dcfPiukuRIENlgV6HzmdBh/BH1V8jn5s31DwR9VfIi3Uy7Q+wMSSZAkGJ79l7Ir4p2pU21xb3RXpb+SuzDnI1MTlshJrXOXC1LryPEbfLexJYuqrp+Si+/Lg0v6U+b+Huv1Gycl0qdpV35WXJXUF9X7d3Q6ilTjCKjCKjFbkkkkl0SKqo2m1EVsWvH2/WhbU2y3KqOm04e/71MvJx81e5AzBR2L/EoABkwAAAAAAAAAAAAAAAAAAAAADyY3CQr05UqsFOElZp/e59VoesGUW2aGFS5S+bsqVMDLtxvOjJ92fGLekZW49dH8DmZH6Jr0Y1IyhOKlGStKLSaafBoqjNmRqlCTq4SMqlJ7+yrynDo1q1yevPS7vKOvR+5Jrx4/PqUNZs/AuOLThw+PTu04mR0FDwR9VfI1FHBSb73dSe++vosdfsnLmIxFnGn2YefK6jbpxfsRYuckaYnrZOZVsasq4WJdeRqrmz2TsKvifw6fd897o/8AfsudvsnJ9CjaVReVmuMl3U+kNPedJpuRWT7VRMokvzX9Jr527i2p9kqucy+CftdPK/JUOX2Rk6jStKt/Fn1VoL/54+2/oOmpwUUkkkluSSskuiMwVEs0kq3et+utC4ihjiSzEt128fEAA1m0AAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAAGUNfX/Hj7DYAEndncZ494ABEiAAAAAAAAAf/2Q==",
        xref="paper", yref="paper",
        x=0.7, y=0.43,
        sizex=0.1, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)
fig.add_layout_image(
    dict(
        source="https://avatars.githubusercontent.com/u/22800682?s=280&v=4",
        xref="paper", yref="paper",
        x=0.6, y=0.09,
        sizex=0.1, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)

fig.show()

#### **Data Visualisation Packages**

This order may be due to the fact that:

* **Matplotlib** is a good choice for data scientists who need complete control over the appearance of their graphs and allows for a wide variety of graphs. 
* **Seaborn** is a good choice for data scientists looking for an easy way to create statistical plots. 
* **Plotly** is a good choice for data scientists who need to create interactive plots and is built on top of Matplotlib.


In [ ]:
i=15

tabla = value_count_from_multiple_choice(df_prue, v_v1,i)[-3:]
name='Data Visualisation Packages'
color = ['rgb(199,233,192)']*(tabla.shape[0]-3) + ['rgb(0,68,27)']*3
fig = go.Figure()
fig.add_trace(go.Bar(x=tabla["% Users"],   y=tabla.index,   marker_color=color,name= '',
                    text =tabla["Yes"].values,texttemplate = "%{x}%<br>%{text}",textposition = "outside", orientation='h'))
fig.update_traces(textposition='inside', textfont_size=500)
fig.update_xaxes(title_text="% ")
fig.update_layout(barmode='overlay',title_text=name,xaxis_range=[0, 95],    height=500, width=900,)

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/8/84/Matplotlib_icon.svg",
        # xref="paper", yref="paper",
        x=0.99, y=0.73,
        sizex=0.1, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)
fig.add_layout_image(
    dict(
        source="https://seaborn.pydata.org/_images/logo-mark-lightbg.svg",
        xref="paper", yref="paper",
        x=0.89, y=0.4,
        sizex=0.1, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/8/8a/Plotly-logo.png",
        xref="paper", yref="paper",
        x=0.9, y=0.03,
        sizex=0.4, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)
fig.show()

#### **Cloud computing  platforms**

Some specific factors that may influence the decision to use a particular cloud platform:

* **AWS:** AWS is the most mature cloud platform and has the broadest range of services available. It is a good choice for companies that need a cloud platform that can handle complex workloads and has a broad feature set.
* **GCP:** GCP is a relatively new cloud platform, but it has grown rapidly in popularity. It is a good choice for companies looking for a cloud platform that is affordable and easy to use. GCP also has a strong focus on innovation and offers a number of cutting-edge services.
* **Azure:** Azure is a cloud platform that offers a wide range of services, including integration with Microsoft enterprise systems. It is a good option for companies already using other Microsoft products or services.

In [ ]:
i=31

tabla = value_count_from_multiple_choice(df_prue, v_v1,i)[-3:]
name='Cloud computing  platforms '
color = ['rgb(199,233,192)']*(tabla.shape[0]-3) + ['rgb(0,68,27)']*3
fig = go.Figure()
fig.add_trace(go.Bar(x=tabla["% Users"],   y=tabla.index,   marker_color=color,name= '',
                    text =tabla["Yes"].values,texttemplate = "%{x}%<br>%{text}",textposition = "outside", orientation='h'))
fig.update_traces(textposition='inside', textfont_size=500)
fig.update_xaxes(title_text="% ")
fig.update_layout(barmode='overlay',title_text=name,xaxis_range=[0, 53],    height=500, width=900,)

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/9/93/Amazon_Web_Services_Logo.svg",
        # xref="paper", yref="paper",
        x=0.99, y=0.73,
        sizex=0.1, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)
fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/5/51/Google_Cloud_logo.svg",
        xref="paper", yref="paper",
        x=0.95, y=0.45,
        sizex=0.3, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/a/a8/Microsoft_Azure_Logo.svg",
        xref="paper", yref="paper",
        x=0.7, y=0.1,
        sizex=0.2, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)

fig.show()

#### **Databases**

* **MySQL** is free, open source and easy to learn. Scalable, which means it can grow as business needs require.
* **PostgreSQL** is an open source relational database that offers a wide range of functions and features. It is a good choice for companies that need a powerful and flexible database. However, PostgreSQL can be more complex to learn and use than MySQL.
* **Microsoft SQL Server** is a proprietary relational database that offers a wide range of functions and features. It is a good choice for companies that need a Microsoft-compatible database. However, Microsoft SQL Server has a licensing cost, which can be a deterrent for companies on a limited budget.

In [ ]:
i=35

tabla = value_count_from_multiple_choice(df_prue, v_v1,i)[-3:]
name='Databases'
color = ['rgb(199,233,192)']*(tabla.shape[0]-3) + ['rgb(0,68,27)']*3
fig = go.Figure()
fig.add_trace(go.Bar(x=tabla["% Users"],   y=tabla.index,   marker_color=color,name= '',
                    text =tabla["Yes"].values,texttemplate = "%{x}%<br>%{text}",textposition = "outside", orientation='h'))
fig.update_traces(textposition='inside', textfont_size=500,textangle=0)
fig.update_xaxes(title_text="% ")
fig.update_layout(barmode='overlay',title_text=name,xaxis_range=[0, 32],    height=500, width=900,)

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/0/0a/MySQL_textlogo.svg",
        xref="paper", yref="paper",
        x=0.98, y=0.8,
        sizex=0.1, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)
fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/2/29/Postgresql_elephant.svg",
        xref="paper", yref="paper",
        x=0.9, y=0.42,
        sizex=0.07, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/9/99/Logo_M_SQL_Server.png",
        xref="paper", yref="paper",
        x=0.8, y=0.1,
        sizex=0.2, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)

fig.show()

#### **Business intelligence tools**

The next bar chart presents a comparison between popular business intelligence tools: 

* **Tableau** is widely recognized for its ease of use and its ability to generate interactive and engaging visualizations. Its wide variety of features and customization options make it a popular tool among data professionals and business analysts looking to present data in an effective and accessible way.
* **None** represents users who choose not to use any specific business intelligence tool. This choice could be due to different reasons, such as the use of custom in-house tools, preference for analyzing data directly with programming languages, or perhaps because some users are not directly involved in data visualization and analysis.
* **Microsoft Power BI** is a robust tool that is well integrated with the Microsoft ecosystem, making it attractive to those who already use Microsoft products in their business operations. Its ease of integration with other Microsoft tools and its focus on business data analysis may explain its popularity.

In [ ]:
i=36

tabla = value_count_from_multiple_choice(df_prue, v_v1,i)[-3:]
name='Business intelligence tools'
color = ['rgb(199,233,192)']*(tabla.shape[0]-3) + ['rgb(0,68,27)']*3
fig = go.Figure()
fig.add_trace(go.Bar(x=tabla["% Users"],   y=tabla.index,   marker_color=color,name= '',
                    text =tabla["Yes"].values,texttemplate = "%{x}%<br>%{text}",textposition = "outside", orientation='h'))
fig.update_traces(textposition='inside', textfont_size=500)
fig.update_xaxes(title_text="% ")
fig.update_layout(barmode='overlay',title_text=name,xaxis_range=[0, 36],    height=500, width=900,)

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/4/4b/Tableau_Logo.png",
        xref="paper", yref="paper",
        x=0.98, y=0.8,
        sizex=0.15, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/c/cf/New_Power_BI_Logo.svg",
        xref="paper", yref="paper",
        x=0.72, y=0.1,
        sizex=0.07, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)

fig.show()

#### **Machine Learning Frameworks**

Scikit-learn, XGBoost and TensorFlow are all popular machine learning algorithms, but they have their own strengths and weaknesses. Choosing the right algorithm depends on the specific needs of your application.

* **Scikit-learn** is an open source machine learning toolkit that is easy to use and has a wide selection of algorithms. It is a good choice for machine learning beginners and for applications that require flexibility and control.
* **XGBoost** is a decision tree-based machine learning algorithm that is very accurate and scalable. It is a good choice for applications that require high accuracy and can handle large amounts of data.
* **TensorFlow** is an open source machine learning framework that is powerful and flexible. It is a good choice for applications that require complex models or need to be trained on big data.


In [ ]:
i=17

tabla = value_count_from_multiple_choice(df_prue, v_v1,i)[-3:]
name='Machine Learning Frameworks'
color = ['rgb(199,233,192)']*(tabla.shape[0]-3) + ['rgb(0,68,27)']*3
fig = go.Figure()
fig.add_trace(go.Bar(x=tabla["% Users"],   y=tabla.index,   marker_color=color,name= '',
                    text =tabla["Yes"].values,texttemplate = "%{x}%<br>%{text}",textposition = "outside", orientation='h'))
fig.update_traces(textposition='inside', textfont_size=500)
fig.update_xaxes(title_text="% ")
fig.update_layout(barmode='overlay',title_text=name,xaxis_range=[0, 93],    height=500, width=900,)
fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/0/05/Scikit_learn_logo_small.svg",
        xref="paper", yref="paper",
        x=0.97, y=0.74,
        sizex=0.15, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)
fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/6/69/XGBoost_logo.png",
        xref="paper", yref="paper",
        x=0.73, y=0.415,
        sizex=0.15, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)
fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/a/ab/TensorFlow_logo.svg",
        xref="paper", yref="paper",
        x=0.58, y=0.09,
        sizex=0.13, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)
fig.update_xaxes(range=[0, 110])
fig.show()

#### **Machine Learning algorithms**

The bar chart shows a comparison of popular machine learning algorithms:

* **Linear Regression** is a simple and easy to implement model, especially suitable for regression problems where continuous numerical values are to be predicted. On the other hand, Logistic Regression is widely used in binary classification problems, where one of two possible classes is to be predicted. Its efficiency, interpretability and ability to deal with linear problems make it an attractive option for many users.
* **Decision Trees** are nonlinear models used for both classification and regression problems. They are known for their ease of interpretation and visualization, as they are represented as trees with decision nodes and leaves representing the predictions. Random Forests are an extension of Decision Trees that combine multiple trees to improve accuracy and reduce overfitting in the model.
* **Gradient Boosting Machines** are a set of algorithms that are widely used in supervised learning problems. These algorithms work in sequential iterations to improve model accuracy by correcting errors made by previous models. xgboost and lightgbm are popular examples of Gradient Boosting libraries that offer exceptional performance and training speed.

In [ ]:
i=18

tabla = value_count_from_multiple_choice(df_prue, v_v1,i)[-3:]
tabla.index=name_column_graph(tabla,'(')
name='Machine Learning algorithms'
color = ['rgb(199,233,192)']*(tabla.shape[0]-3) + ['rgb(0,68,27)']*3
fig = go.Figure()
fig.add_trace(go.Bar(x=tabla["% Users"],   y=tabla.index,   marker_color=color,name= '',
                    text =tabla["Yes"].values,texttemplate = "%{x}%<br>%{text}",textposition = "outside", orientation='h'))
fig.update_traces(textposition='inside', textfont_size=500)
fig.update_xaxes(title_text="% ")
fig.update_layout(barmode='overlay',title_text=name,xaxis_range=[0, 90],    height=500, width=900,)
fig.show()

#### **Favorite media sources on data science topics**

* **Blogs**, such as Towards and Analytics Vidhya, are the most widely used media source for their detailed content, practical tutorials and in-depth approach to various data science topics.
* **Kaggle**, with its notebooks and forums, users turn to this platform for data science information and resources. Kaggle is known for its active community, data science competitions and abundant content shared by professionals and enthusiasts in the field.
* **YouTube**, with channels such as Kaggle YouTube and Cloud AI Adventures, YouTube offers instructional videos, visual tutorials and inspirational talks, making it an attractive and accessible source for many users.

In summary, the bar chart highlights the most popular media sources used by users to obtain data science information. Blogs are the preferred choice, followed closely by Kaggle and YouTube.

In [ ]:
i=44

tabla = value_count_from_multiple_choice(df_prue, v_v1,i)[-3:]
tabla.index=name_column_graph(tabla,'(')
name='Favorite media sources on data science topics'
color = ['rgb(199,233,192)']*(tabla.shape[0]-3) + ['rgb(0,68,27)']*3
fig = go.Figure()
fig.add_trace(go.Bar(x=tabla["% Users"],   y=tabla.index,   marker_color=color,name= '',
                    text =tabla["Yes"].values,texttemplate = "%{x}%<br>%{text}",textposition = "outside", orientation='h'))
fig.update_traces(textposition='inside', textfont_size=500)
fig.update_xaxes(title_text="% ")
fig.update_layout(barmode='overlay',title_text=name,xaxis_range=[0, 68],    height=500, width=900,)

fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/7/7c/Kaggle_logo.png",
        xref="paper", yref="paper",
        x=0.95, y=0.415,
        sizex=0.15, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)
fig.add_layout_image(
    dict(
        source="https://upload.wikimedia.org/wikipedia/commons/0/09/YouTube_full-color_icon_%282017%29.svg",
        xref="paper", yref="paper",
        x=0.91, y=0.06,
        sizex=0.13, sizey=1,
        xanchor="right", yanchor="bottom", sizing= "contain",
    )
)

fig.show()

<a id='conclusions'></a>
## **Conclusions**

The demographic, technological, and data science techniques comparison among the top 5 countries most present on Kaggle underscores the significance of fostering data science growth in these nations, particularly within the third-ranked country. Emphasizing the need for promoting gender inclusivity and equitable representation within the field is crucial to achieve greater diversity and enrich the data science community with varied perspectives. The diversity in age ranges also contributes to enriching the community by incorporating professionals from different generations.

Furthermore, the data reveals a high adoption of machine learning techniques in the business realm, signaling their increasing importance across diverse industries. It's important to consider that salary variations among countries may stem from economic factors and specific hiring policies unique to each location.

The presented graphs and analyses empower informed decision-making regarding suitable technologies and tools within the data science domain. Additionally, the availability of resources enhances learning and professional development in this field, providing users with access to valuable information. In conclusion, this study highlights the relevance and potential of data science in these countries, offering insights to continue driving their growth and advancement.


## **Challenges**

Selecting the work objective is undoubtedly one of the most significant challenges when carrying out a data analysis project. A clear and well-defined objective is essential to properly focus the research and ensure that the conclusions are relevant and useful. If the objective is too broad, obtaining concrete and meaningful results can be difficult, potentially complicating data interpretation and informed decision-making.

Likewise, choosing the appropriate questions to showcase the most used tools also presents a significant challenge. It's crucial to pose questions that are relevant to the analysis and provide valuable insights into the landscape of tools in the field of data science. A proper selection of questions will ensure that graphs and analyses accurately and comprehensively reflect preferences and trends in technology usage within the field.

On the other hand, creating effective graphs that clearly display the expected results may require additional time and effort. Technical challenges in data manipulation and visual presentation are common. It's important to consider aspects such as choosing suitable charts, colors, and labels to achieve an accurate and appealing representation of the data.

In summary, encountering challenges during data analysis is a natural part of the process. Through experience and practice, it's possible to enhance the selection of objectives, questions, and graphical techniques, resulting in more accurate and meaningful outcomes in future data analysis projects.



